# Working with async

`openaivec.aio` is a experimental module that provides an async interface to OpenAI's API. It is designed to be used with Python's `asyncio` library, allowing you to make non-blocking requests to the OpenAI API.

In [1]:
import os
from openai import AsyncOpenAI
from openaivec.aio import pandas_ext
import pandas as pd
from pydantic import BaseModel, Field
from enum import Enum

pandas_ext.use(AsyncOpenAI())
pandas_ext.responses_model("gpt-4.1-mini")
pandas_ext.embeddings_model("text-embedding-3-small")

The model name 'gpt-4.1-mini' is not supported by tiktoken. Instead, using the 'o200k_base' encoding.


In [2]:
files_dict = {"path": []}
for root, dirs, files in os.walk('../../src/openaivec'):
    for file in files:
        path = os.path.join(root, file)
        if "__" not in path:
            files_dict["path"].append(os.path.join(root, file))

implements_df = pd.DataFrame(files_dict).assign(
    module=lambda df: df["path"].str.split("/")
        .map(lambda x: x[3:])
        .map(lambda x: ".".join(x))
        .map(lambda x: x.replace(".py", "")),
)

In [3]:
implements_df

,path,module
0,../../src/openaivec/serialize.py,openaivec.serialize
1,../../src/openaivec/responses.py,openaivec.responses
2,../../src/openaivec/log.py,openaivec.log
3,../../src/openaivec/util.py,openaivec.util
4,../../src/openaivec/embeddings.py,openaivec.embeddings
5,../../src/openaivec/pandas_ext.py,openaivec.pandas_ext
6,../../src/openaivec/spark.py,openaivec.spark
7,../../src/openaivec/prompt.py,openaivec.prompt
8,../../src/openaivec/beta/batch.py,openaivec.beta.batch
9,../../src/openaivec/beta/schema.py,openaivec.beta.schema


In [4]:
class OpjectType(str, Enum):
    FUNCTION = "function"
    CLASS = "class"
    VARIABLE = "variable"

class Question(BaseModel):
    question: str = Field(description="The specific question related to the code section.")
    answer: str = Field(description="The corresponding answer explaining the code aspect.")

class Section(BaseModel):
    name: str = Field(description="The name of the function or class being documented.")
    type: OpjectType = Field(description="The type of the code section, either a function or a class.")
    description: str = Field(description="A concise summary of the function or class's purpose and functionality.")
    questions: list[Question] = Field(description="A list of Q&A pairs clarifying aspects of this code section.")

class Document(BaseModel):
    sections: list[Section] = Field(description="A list of sections, each documenting a specific function or class.")

Note that async methods are not available in lambda functions.

In [5]:
codes_df = implements_df.assign(
        code=lambda df: df['path'].apply(lambda x: open(x).read()),
)

docs_df = codes_df.assign(
    doc=await codes_df["code"].ai.responses(
        instructions="Generate a detailed documentation for the code, including sections for each function or class. Each section should include a name, description, and a list of questions and answers that clarify the code's functionality.",
        response_format=Document,
        batch_size=1,
    )
).pipe(
    lambda df: df.drop(columns=["code", "path"], errors="ignore")
)


In [6]:
docs_df

,module,doc
0,openaivec.serialize,"sections=[Section(name='serialize_base_model',..."
1,openaivec.responses,sections=[Section(name='_vectorize_system_mess...
2,openaivec.log,"sections=[Section(name='observe', type=<Opject..."
3,openaivec.util,"sections=[Section(name='split_to_minibatch', t..."
4,openaivec.embeddings,"sections=[Section(name='VectorizedEmbeddings',..."
5,openaivec.pandas_ext,"sections=[Section(name='use', type=<OpjectType..."
6,openaivec.spark,"sections=[Section(name='UDFBuilder', type=<Opj..."
7,openaivec.prompt,"sections=[Section(name='Example', type=<Opject..."
8,openaivec.beta.batch,"sections=[Section(name='JsonlUDFBuilder', type..."
9,openaivec.beta.schema,"sections=[Section(name='Entity', type=<OpjectT..."


In [7]:
questions_df = docs_df.pipe(
    lambda df: df
    .ai.extract("doc")
    .explode("doc_sections")
    .ai.extract("doc_sections")
    .explode("doc_sections_questions")
    .ai.extract("doc_sections_questions")
    .assign(
        doc_sections_type=lambda df: df.doc_sections_type.map(lambda x: x.value),
    )
)

questions_df = questions_df.assign(
    embedding=await questions_df["doc_sections_questions_question"].ai.embeddings()
)

In [8]:
from IPython.display import display, Markdown

display(Markdown(questions_df.to_markdown(index=False)))

| module                   | doc_sections_name             | doc_sections_type   | doc_sections_description                                                                                                                                                                                                                                                                                                                                                                         | doc_sections_questions_question                                                          | doc_sections_questions_answer                                                                                                                                                                                                                                                                                      | embedding                                                               |
|:-------------------------|:------------------------------|:--------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-----------------------------------------------------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:------------------------------------------------------------------------|
| openaivec.serialize      | serialize_base_model          | function            | Serializes a Pydantic BaseModel class into its JSON schema representation using Pydantic's built-in model_json_schema method.                                                                                                                                                                                                                                                                    | What does serialize_base_model do?                                                       | It takes a Pydantic BaseModel class and returns its JSON schema as a dictionary.                                                                                                                                                                                                                                   | [ 0.00703712  0.02428696  0.00814858 ...  0.01711011 -0.01465855        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.00564939]                                                           |
| openaivec.serialize      | serialize_base_model          | function            | Serializes a Pydantic BaseModel class into its JSON schema representation using Pydantic's built-in model_json_schema method.                                                                                                                                                                                                                                                                    | Does serialize_base_model serialize an instance or a class?                              | It serializes the BaseModel class itself, not an instance.                                                                                                                                                                                                                                                         | [ 0.01213912  0.00542506 -0.00110337 ...  0.0178444  -0.00793583        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.00294791]                                                           |
| openaivec.serialize      | dereference_json_schema       | function            | Recursively resolves all JSON Schema $ref references within a JSON schema dictionary by replacing them with their actual definitions from the $defs section.                                                                                                                                                                                                                                     | What is the purpose of dereference_json_schema?                                          | To replace all $ref references in a JSON schema with the actual referenced definitions, producing a fully dereferenced schema.                                                                                                                                                                                     | [ 0.04457856  0.02287264  0.02154792 ...  0.02960561 -0.01883772        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.00619214]                                                           |
| openaivec.serialize      | dereference_json_schema       | function            | Recursively resolves all JSON Schema $ref references within a JSON schema dictionary by replacing them with their actual definitions from the $defs section.                                                                                                                                                                                                                                     | How does dereference_json_schema handle nested references?                               | It recursively traverses dictionaries and lists, dereferencing any $ref it encounters until all references are resolved.                                                                                                                                                                                           | [ 0.03085474 -0.00038323  0.05200343 ...  0.02277827  0.01143076        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.02145796]                                                           |
| openaivec.serialize      | dereference_json_schema       | function            | Recursively resolves all JSON Schema $ref references within a JSON schema dictionary by replacing them with their actual definitions from the $defs section.                                                                                                                                                                                                                                     | What happens to the $defs key in the output?                                             | The $defs key and its contents are omitted from the returned dereferenced schema.                                                                                                                                                                                                                                  | [0.00867954 0.01348657 0.08056984 ... 0.00212451 0.00995021 0.00438683] |
| openaivec.serialize      | parse_field                   | function            | Parses a JSON schema field definition and returns the corresponding Python type or Pydantic model type, supporting primitive types, objects, arrays, and enums.                                                                                                                                                                                                                                  | How does parse_field handle primitive JSON schema types?                                 | It maps 'string' to str, 'integer' to int, 'number' to float, and 'boolean' to bool.                                                                                                                                                                                                                               | [-0.0002183   0.02442133  0.002171   ...  0.02539187 -0.02931914        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.03347212]                                                           |
| openaivec.serialize      | parse_field                   | function            | Parses a JSON schema field definition and returns the corresponding Python type or Pydantic model type, supporting primitive types, objects, arrays, and enums.                                                                                                                                                                                                                                  | How are object types handled in parse_field?                                             | It recursively calls deserialize_base_model to create a Pydantic model for the object type.                                                                                                                                                                                                                        | [-0.00945058  0.03055124 -0.03717389 ...  0.01455051  0.00071416        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.03412844]                                                           |
| openaivec.serialize      | parse_field                   | function            | Parses a JSON schema field definition and returns the corresponding Python type or Pydantic model type, supporting primitive types, objects, arrays, and enums.                                                                                                                                                                                                                                  | How does parse_field handle arrays?                                                      | It parses the 'items' field to determine the inner type and returns a List of that type.                                                                                                                                                                                                                           | [-0.01200029 -0.02698431 -0.01803313 ...  0.01716535 -0.0189009         |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.03609003]                                                           |
| openaivec.serialize      | parse_field                   | function            | Parses a JSON schema field definition and returns the corresponding Python type or Pydantic model type, supporting primitive types, objects, arrays, and enums.                                                                                                                                                                                                                                  | What happens if parse_field encounters an unsupported type?                              | It raises a ValueError indicating the unsupported type.                                                                                                                                                                                                                                                            | [-0.02944409  0.01313342  0.01184191 ...  0.02626684 -0.02420527        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.04877435]                                                           |
| openaivec.serialize      | deserialize_base_model        | function            | Dynamically creates a Pydantic BaseModel class from a JSON schema dictionary by parsing its properties, handling enums, nested objects, and arrays.                                                                                                                                                                                                                                              | How does deserialize_base_model create the model fields?                                 | It dereferences the JSON schema, then iterates over the 'properties' to define fields with types parsed by parse_field or dynamically created enums.                                                                                                                                                               | [-0.0098351   0.04046241  0.0286112  ...  0.01764825 -0.00840922        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.00291021]                                                           |
| openaivec.serialize      | deserialize_base_model        | function            | Dynamically creates a Pydantic BaseModel class from a JSON schema dictionary by parsing its properties, handling enums, nested objects, and arrays.                                                                                                                                                                                                                                              | How are enum fields handled?                                                             | For fields with an 'enum' key, it creates a dynamic Enum class with the given title and enum values, and uses that as the field type.                                                                                                                                                                              | [ 0.02811901  0.01801879  0.02084685 ... -0.04118196 -0.01253774        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.05095897]                                                           |
| openaivec.serialize      | deserialize_base_model        | function            | Dynamically creates a Pydantic BaseModel class from a JSON schema dictionary by parsing its properties, handling enums, nested objects, and arrays.                                                                                                                                                                                                                                              | What is the return value of deserialize_base_model?                                      | It returns a dynamically created Pydantic BaseModel subclass named after the 'title' in the JSON schema.                                                                                                                                                                                                           | [-0.0020762   0.04951167  0.00817674 ...  0.02675581  0.00246491        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.01164622]                                                           |
| openaivec.responses      | _vectorize_system_message     | function            | Generates a composite system prompt that instructs the language model to handle multiple user messages in a single batch. It embeds the original system message and adds XML-like instructions explaining how the model should process an array of JSON user messages and produce corresponding outputs in order.                                                                                | What is the purpose of the _vectorize_system_message function?                           | It creates a special system prompt that teaches the language model how to process multiple user messages sent as a batch in JSON format and how to return responses in the same order.                                                                                                                             | [ 0.01709459  0.05447612 -0.0606034  ...  0.01939875 -0.02363378        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.02018397]                                                           |
| openaivec.responses      | _vectorize_system_message     | function            | Generates a composite system prompt that instructs the language model to handle multiple user messages in a single batch. It embeds the original system message and adds XML-like instructions explaining how the model should process an array of JSON user messages and produce corresponding outputs in order.                                                                                | What does the returned prompt contain?                                                   | It contains the original system message wrapped inside XML-like tags, additional instructions for batch processing, and an example input-output pair demonstrating the expected JSON structure for multiple messages.                                                                                              | [ 0.02785004  0.01672135  0.02523069 ... -0.0222857   0.01039952        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.02196005]                                                           |
| openaivec.responses      | Message                       | class               | A generic Pydantic model representing a message with an integer ID and a body of generic type T. Used to encapsulate user or assistant messages with associated IDs for ordering.                                                                                                                                                                                                                | What is the purpose of the Message class?                                                | It models a message with an ID and a body, where the body can be of any type, allowing consistent handling of user and assistant messages with unique identifiers.                                                                                                                                                 | [ 0.00056533  0.02341876 -0.00888275 ...  0.0100264  -0.02304187        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.00351866]                                                           |
| openaivec.responses      | Request                       | class               | A Pydantic model representing a request containing a list of user messages, each wrapped in a Message[str] object. This structure is used to send multiple user prompts in a single API call.                                                                                                                                                                                                    | What does the Request class represent?                                                   | It represents the input payload to the language model, containing multiple user messages each with an ID and string body.                                                                                                                                                                                          | [-0.0218132   0.01502862 -0.01244496 ...  0.01421342  0.0092236         |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.00130662]                                                           |
| openaivec.responses      | Response                      | class               | A generic Pydantic model representing the response from the language model, containing a list of assistant messages each wrapped in a Message[T] object. The generic type T allows flexibility in the response body type.                                                                                                                                                                        | What is the role of the Response class?                                                  | It models the output from the language model, holding multiple assistant messages with their IDs and bodies of generic type T.                                                                                                                                                                                     | [-0.04568828  0.03900085  0.00084064 ...  0.01806153 -0.00765354        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.01014077]                                                           |
| openaivec.responses      | VectorizedResponses           | class               | An abstract base class defining the interface for batched language model interactions. It declares an abstract method parse that takes a list of input strings and a batch size, returning a list of outputs in the original order.                                                                                                                                                              | What is the purpose of the VectorizedResponses class?                                    | It defines a minimal interface for classes that implement batched language model calls, ensuring they provide a parse method to process multiple inputs efficiently.                                                                                                                                               | [-0.01053904  0.05806106 -0.00999558 ...  0.0415508  -0.02758588        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.00192276]                                                           |
| openaivec.responses      | VectorizedResponses           | class               | An abstract base class defining the interface for batched language model interactions. It declares an abstract method parse that takes a list of input strings and a batch size, returning a list of outputs in the original order.                                                                                                                                                              | What does the parse method do?                                                           | It accepts a list of user prompt strings and a batch size, and returns a list of model responses in the same order as the inputs.                                                                                                                                                                                  | [-0.02220551  0.00792778 -0.00546444 ...  0.00282238 -0.02663631        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.00561578]                                                           |
| openaivec.responses      | VectorizedResponsesOpenAI     | class               | A stateless, generic class that wraps the OpenAI JSON-mode API to enable sending multiple user prompts in a single request and receiving ordered responses. It supports batching, optional parallel execution, and automatic de-duplication of identical prompts to optimize token usage.                                                                                                        | What is the main functionality of VectorizedResponsesOpenAI?                             | It provides a batched interface to OpenAI's JSON-mode API, allowing multiple prompts to be sent and responses received in order, with support for batching and parallelism.                                                                                                                                        | [-0.009904    0.05326656  0.02927791 ...  0.02704734 -0.01604141        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.00144586]                                                           |
| openaivec.responses      | VectorizedResponsesOpenAI     | class               | A stateless, generic class that wraps the OpenAI JSON-mode API to enable sending multiple user prompts in a single request and receiving ordered responses. It supports batching, optional parallel execution, and automatic de-duplication of identical prompts to optimize token usage.                                                                                                        | What are the key attributes of VectorizedResponsesOpenAI?                                | Attributes include the OpenAI client, model name, system message, temperature, top_p, response format type, and a flag for parallel execution.                                                                                                                                                                     | [-0.01390528  0.05879487  0.01632593 ...  0.03095741 -0.00488837        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.01547871]                                                           |
| openaivec.responses      | VectorizedResponsesOpenAI     | class               | A stateless, generic class that wraps the OpenAI JSON-mode API to enable sending multiple user prompts in a single request and receiving ordered responses. It supports batching, optional parallel execution, and automatic de-duplication of identical prompts to optimize token usage.                                                                                                        | How does VectorizedResponsesOpenAI ensure the model understands batch inputs?            | It uses the _vectorize_system_message function to create a special system prompt that instructs the model on how to handle multiple user messages and produce corresponding outputs.                                                                                                                               | [-0.01567682  0.03388376  0.01237774 ...  0.02654145 -0.00240454        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.01593727]                                                           |
| openaivec.responses      | VectorizedResponsesOpenAI     | class               | A stateless, generic class that wraps the OpenAI JSON-mode API to enable sending multiple user prompts in a single request and receiving ordered responses. It supports batching, optional parallel execution, and automatic de-duplication of identical prompts to optimize token usage.                                                                                                        | What is the role of the _request_llm method?                                             | It performs a single OpenAI API call in JSON mode with a batch of user messages, parses the response into a structured format, and handles rate limiting with exponential backoff.                                                                                                                                 | [-0.00288399  0.0033851   0.01683437 ... -0.02468386  0.00614293        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.0303327 ]                                                           |
| openaivec.responses      | VectorizedResponsesOpenAI     | class               | A stateless, generic class that wraps the OpenAI JSON-mode API to enable sending multiple user prompts in a single request and receiving ordered responses. It supports batching, optional parallel execution, and automatic de-duplication of identical prompts to optimize token usage.                                                                                                        | How does _predict_chunk work?                                                            | It converts a list of user prompt strings into Message objects with stable IDs, calls _request_llm to get responses, and then reorders the responses to match the original input order.                                                                                                                            | [ 0.008925   -0.02296053 -0.02212515 ... -0.02368534 -0.0100675         |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.00380833]                                                           |
| openaivec.responses      | VectorizedResponsesOpenAI     | class               | A stateless, generic class that wraps the OpenAI JSON-mode API to enable sending multiple user prompts in a single request and receiving ordered responses. It supports batching, optional parallel execution, and automatic de-duplication of identical prompts to optimize token usage.                                                                                                        | What does the parse method do?                                                           | It is the public API that accepts all input prompts and a batch size, de-duplicates inputs, splits them into minibatches, and processes them either serially or in parallel using _predict_chunk, returning responses in the original input order.                                                                 | [-0.02220551  0.00792778 -0.00546444 ...  0.00282238 -0.02663631        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.00561578]                                                           |
| openaivec.responses      | VectorizedResponsesOpenAI     | class               | A stateless, generic class that wraps the OpenAI JSON-mode API to enable sending multiple user prompts in a single request and receiving ordered responses. It supports batching, optional parallel execution, and automatic de-duplication of identical prompts to optimize token usage.                                                                                                        | How does the class handle duplicate prompts?                                             | Duplicates are de-duplicated internally to avoid redundant calls, improving efficiency, but the final output preserves the original input order including duplicates.                                                                                                                                              | [ 0.03218302  0.00174642  0.0571668  ... -0.016061   -0.01964537        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.01973689]                                                           |
| openaivec.log            | observe                       | function            | A decorator factory that creates a decorator to log the start and end of method calls within a class. It uses a provided Logger instance to create child loggers specific to the class and method names, logs a unique transaction ID, timestamps, and the event type ('start' or 'end') in JSON format.                                                                                         | What is the purpose of the observe function?                                             | The observe function is a decorator factory that returns a decorator to wrap class methods, enabling logging of method invocation start and end events with detailed contextual information.                                                                                                                       | [ 0.00277905  0.03274433 -0.04986133 ... -0.01011139 -0.02127679        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.02102383]                                                           |
| openaivec.log            | observe                       | function            | A decorator factory that creates a decorator to log the start and end of method calls within a class. It uses a provided Logger instance to create child loggers specific to the class and method names, logs a unique transaction ID, timestamps, and the event type ('start' or 'end') in JSON format.                                                                                         | How does the decorator log the start and end of a method call?                           | It logs a JSON-formatted message before the method execution indicating the start, including a unique transaction ID, class name, method name, and timestamp. After the method completes (even if an exception occurs), it logs a similar message indicating the end of the method call.                           | [ 0.03206281  0.02017103  0.02172661 ... -0.0328856  -0.01457868        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.02116094]                                                           |
| openaivec.log            | observe                       | function            | A decorator factory that creates a decorator to log the start and end of method calls within a class. It uses a provided Logger instance to create child loggers specific to the class and method names, logs a unique transaction ID, timestamps, and the event type ('start' or 'end') in JSON format.                                                                                         | What is the role of the transaction_id in the logs?                                      | The transaction_id is a unique UUID string generated for each method call to correlate the start and end log entries for the same invocation.                                                                                                                                                                      | [-0.01762005  0.0067519  -0.01328329 ... -0.00216969  0.01564593        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.00634238]                                                           |
| openaivec.log            | observe                       | function            | A decorator factory that creates a decorator to log the start and end of method calls within a class. It uses a provided Logger instance to create child loggers specific to the class and method names, logs a unique transaction ID, timestamps, and the event type ('start' or 'end') in JSON format.                                                                                         | How are the logger instances structured?                                                 | The decorator creates a child logger from the provided logger using the class name, then further creates a child logger from that using the method name, allowing hierarchical and contextual logging.                                                                                                             | [-0.00484941  0.00160006  0.03589756 ... -0.01211121 -0.0008475         |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.0081937 ]                                                           |
| openaivec.log            | observe                       | function            | A decorator factory that creates a decorator to log the start and end of method calls within a class. It uses a provided Logger instance to create child loggers specific to the class and method names, logs a unique transaction ID, timestamps, and the event type ('start' or 'end') in JSON format.                                                                                         | Why is functools.wraps used in the decorator?                                            | functools.wraps preserves the metadata of the original function (such as its name and docstring) when it is wrapped by the decorator, which is important for introspection and debugging.                                                                                                                          | [ 0.02345033 -0.00920716 -0.02813602 ... -0.0251363  -0.00475959        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.00312288]                                                           |
| openaivec.log            | observe                       | function            | A decorator factory that creates a decorator to log the start and end of method calls within a class. It uses a provided Logger instance to create child loggers specific to the class and method names, logs a unique transaction ID, timestamps, and the event type ('start' or 'end') in JSON format.                                                                                         | What arguments does the decorated function accept?                                       | The decorated function accepts self (the instance of the class), any positional arguments (*args), and keyword arguments (**kwargs), forwarding them to the original method.                                                                                                                                       | [ 0.04615999  0.01306623  0.00796267 ... -0.01816289  0.01379827        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.00582179]                                                           |
| openaivec.log            | observe                       | function            | A decorator factory that creates a decorator to log the start and end of method calls within a class. It uses a provided Logger instance to create child loggers specific to the class and method names, logs a unique transaction ID, timestamps, and the event type ('start' or 'end') in JSON format.                                                                                         | What happens if the decorated method raises an exception?                                | The 'end' log entry is still recorded in the finally block, ensuring that the end of the method call is logged regardless of whether the method completes successfully or raises an exception.                                                                                                                     | [ 0.01341289  0.00327346  0.05865427 ... -0.00577801 -0.01653959        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.00794436]                                                           |
| openaivec.util           | split_to_minibatch            | function            | Splits a list into smaller sublists (mini-batches) each with a maximum specified size. Useful for processing large lists in manageable chunks.                                                                                                                                                                                                                                                   | What does split_to_minibatch do?                                                         | It divides a list into consecutive sublists, each with up to batch_size elements, except possibly the last which may be smaller.                                                                                                                                                                                   | [-0.03423944 -0.02653821  0.0363794  ... -0.00976854 -0.00520461        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.01398903]                                                           |
| openaivec.util           | split_to_minibatch            | function            | Splits a list into smaller sublists (mini-batches) each with a maximum specified size. Useful for processing large lists in manageable chunks.                                                                                                                                                                                                                                                   | Why use split_to_minibatch?                                                              | To process or handle data in smaller batches rather than all at once, which can be more efficient or required by some APIs.                                                                                                                                                                                        | [-0.0202413  -0.02381708  0.04675636 ... -0.03025867  0.00250273        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.01461298]                                                           |
| openaivec.util           | map_minibatch                 | function            | Applies a synchronous function to each mini-batch of a list sequentially and concatenates the results into a single flattened list.                                                                                                                                                                                                                                                              | How does map_minibatch work?                                                             | It splits the input list into mini-batches, applies the function f to each batch, and then flattens the list of results into one list.                                                                                                                                                                             | [-0.05701701 -0.0201699   0.04523728 ... -0.00957587 -0.00412742        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.00834505]                                                           |
| openaivec.util           | map_minibatch                 | function            | Applies a synchronous function to each mini-batch of a list sequentially and concatenates the results into a single flattened list.                                                                                                                                                                                                                                                              | Is the processing parallel?                                                              | No, map_minibatch processes batches sequentially.                                                                                                                                                                                                                                                                  | [-0.03226249  0.004592    0.04064659 ... -0.02017516 -0.03087008        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.03098858]                                                           |
| openaivec.util           | map_minibatch_async           | function            | Asynchronous version of map_minibatch that applies an async function concurrently to mini-batches and gathers the results.                                                                                                                                                                                                                                                                       | What is the difference between map_minibatch and map_minibatch_async?                    | map_minibatch_async supports asynchronous functions and runs batch processing concurrently using asyncio.gather.                                                                                                                                                                                                   | [-0.04313497 -0.03876632  0.03863393 ...  0.00446794 -0.00812504        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.00677363]                                                           |
| openaivec.util           | map_minibatch_async           | function            | Asynchronous version of map_minibatch that applies an async function concurrently to mini-batches and gathers the results.                                                                                                                                                                                                                                                                       | What does it return?                                                                     | A flattened list of results from all batches, preserving order.                                                                                                                                                                                                                                                    | [-0.02937009  0.04300262  0.0378236  ...  0.00201209 -0.01064206        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.01549531]                                                           |
| openaivec.util           | map_minibatch_parallel        | function            | Parallel variant of map_minibatch that uses a thread pool to apply a function to mini-batches concurrently in separate threads.                                                                                                                                                                                                                                                                  | How does map_minibatch_parallel achieve parallelism?                                     | It uses ThreadPoolExecutor to run the function f on batches in parallel threads.                                                                                                                                                                                                                                   | [-0.04255361 -0.01126419  0.05974193 ... -0.00339118 -0.01473285        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.00368619]                                                           |
| openaivec.util           | map_minibatch_parallel        | function            | Parallel variant of map_minibatch that uses a thread pool to apply a function to mini-batches concurrently in separate threads.                                                                                                                                                                                                                                                                  | Is it suitable for CPU-bound or I/O-bound tasks?                                         | It is more suitable for I/O-bound tasks due to Python's GIL limiting CPU-bound thread parallelism.                                                                                                                                                                                                                 | [-0.01219712  0.02421857  0.0445722  ...  0.00559035  0.0109109         |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.00980036]                                                           |
| openaivec.util           | map_unique                    | function            | Applies a function once per unique value in a list and broadcasts the results back to align with the original list order, avoiding redundant computation on duplicates.                                                                                                                                                                                                                          | How does map_unique handle duplicates?                                                   | It extracts unique values, applies the function once to them, then maps results back to the original list positions.                                                                                                                                                                                               | [ 0.01115679 -0.01008725  0.05760149 ... -0.00624948  0.00561684        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.01515534]                                                           |
| openaivec.util           | map_unique                    | function            | Applies a function once per unique value in a list and broadcasts the results back to align with the original list order, avoiding redundant computation on duplicates.                                                                                                                                                                                                                          | Why use map_unique?                                                                      | To optimize processing by avoiding repeated computation on duplicate elements.                                                                                                                                                                                                                                     | [ 0.01279336 -0.00366189  0.0327029  ... -0.01553531 -0.00966483        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.01134006]                                                           |
| openaivec.util           | map_unique_minibatch          | function            | Combines map_unique and map_minibatch to apply a function to unique values in mini-batches, then broadcasts results back to the original list order.                                                                                                                                                                                                                                             | What is the benefit of map_unique_minibatch?                                             | It reduces redundant computation on duplicates and processes unique values efficiently in batches.                                                                                                                                                                                                                 | [-0.02079747 -0.02991777  0.04127862 ... -0.01785841  0.0036277         |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.00619113]                                                           |
| openaivec.util           | map_unique_minibatch          | function            | Combines map_unique and map_minibatch to apply a function to unique values in mini-batches, then broadcasts results back to the original list order.                                                                                                                                                                                                                                             | Is the function f applied to duplicates?                                                 | No, f is applied only to unique values.                                                                                                                                                                                                                                                                            | [ 0.01700726 -0.00663167  0.02252156 ... -0.01196458 -0.00522045        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.03810672]                                                           |
| openaivec.util           | map_unique_minibatch_async    | function            | Asynchronous version of map_unique_minibatch that applies an async function concurrently to mini-batches of unique values and broadcasts results back.                                                                                                                                                                                                                                           | How does map_unique_minibatch_async differ from its synchronous counterpart?             | It supports asynchronous functions and runs batch processing concurrently using asyncio.                                                                                                                                                                                                                           | [-0.04863087 -0.04104864  0.02109477 ...  0.00060536 -0.01512881        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.00870531]                                                           |
| openaivec.util           | map_unique_minibatch_async    | function            | Asynchronous version of map_unique_minibatch that applies an async function concurrently to mini-batches of unique values and broadcasts results back.                                                                                                                                                                                                                                           | What does it return?                                                                     | A list of results aligned with the original input list order.                                                                                                                                                                                                                                                      | [-0.02937009  0.04300262  0.0378236  ...  0.00201209 -0.01064206        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.01549531]                                                           |
| openaivec.util           | map_unique_minibatch_parallel | function            | Parallel version of map_unique_minibatch that applies a function to unique values in mini-batches using parallel threads and broadcasts results back.                                                                                                                                                                                                                                            | How is parallelism achieved in map_unique_minibatch_parallel?                            | By using ThreadPoolExecutor to process batches of unique values in parallel threads.                                                                                                                                                                                                                               | [-0.02136791 -0.02682164  0.04294017 ... -0.01487963  0.0028961         |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.00350278]                                                           |
| openaivec.util           | map_unique_minibatch_parallel | function            | Parallel version of map_unique_minibatch that applies a function to unique values in mini-batches using parallel threads and broadcasts results back.                                                                                                                                                                                                                                            | When should this be used?                                                                | When the function f is thread-safe and benefits from parallel execution.                                                                                                                                                                                                                                           | [0.00708772 0.01030125 0.02903599 ... 0.02729057 0.01281336 0.04352137] |
| openaivec.util           | get_exponential_with_cutoff   | function            | Samples a random value from an exponential distribution with a specified scale, rejecting samples above three times the scale to impose an upper cutoff.                                                                                                                                                                                                                                         | What is the purpose of the cutoff?                                                       | To limit the sampled values to a maximum of three times the scale parameter, avoiding very large outliers.                                                                                                                                                                                                         | [ 0.04096888  0.01484832 -0.0207891  ...  0.00148519 -0.026476          |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.00290873]                                                           |
| openaivec.util           | get_exponential_with_cutoff   | function            | Samples a random value from an exponential distribution with a specified scale, rejecting samples above three times the scale to impose an upper cutoff.                                                                                                                                                                                                                                         | How is the sampling done?                                                                | Repeatedly samples from the exponential distribution until a value less than 3 * scale is obtained.                                                                                                                                                                                                                | [-0.00256591 -0.01752147  0.05685415 ... -0.02022522  0.00104411        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.0059286 ]                                                           |
| openaivec.util           | backoff                       | function            | Decorator that implements exponential back-off retry logic for a function, retrying on a specified exception with randomized delays and optional maximum retries.                                                                                                                                                                                                                                | How does the backoff decorator work?                                                     | It wraps a function to retry it when a specified exception occurs, waiting a random exponentially distributed interval between retries.                                                                                                                                                                            | [ 0.0500652  -0.0197235  -0.0202278  ... -0.02133445 -0.02232902        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.01120654]                                                           |
| openaivec.util           | backoff                       | function            | Decorator that implements exponential back-off retry logic for a function, retrying on a specified exception with randomized delays and optional maximum retries.                                                                                                                                                                                                                                | What happens if max_retries is reached?                                                  | The exception is re-raised and no further retries are attempted.                                                                                                                                                                                                                                                   | [ 0.0142964   0.05617015  0.09593284 ... -0.01348805  0.01062877        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.03762491]                                                           |
| openaivec.util           | backoff                       | function            | Decorator that implements exponential back-off retry logic for a function, retrying on a specified exception with randomized delays and optional maximum retries.                                                                                                                                                                                                                                | What is the role of the scale parameter?                                                 | It controls the scale of the exponential back-off delay between retries.                                                                                                                                                                                                                                           | [0.05551582 0.01386586 0.01402298 ... 0.00037418 0.00442883 0.01804264] |
| openaivec.util           | TextChunker                   | class               | Utility class for splitting text into chunks bounded by a maximum token count, using token-aware sentence segmentation based on specified separators.                                                                                                                                                                                                                                            | What does the split method do?                                                           | It splits text by given separators, then greedily packs sentences into chunks whose token counts do not exceed max_tokens.                                                                                                                                                                                         | [ 0.00221071  0.01504825  0.02271157 ... -0.01076968 -0.04556967        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.00611747]                                                           |
| openaivec.util           | TextChunker                   | class               | Utility class for splitting text into chunks bounded by a maximum token count, using token-aware sentence segmentation based on specified separators.                                                                                                                                                                                                                                            | Why is token count important?                                                            | To ensure chunks fit within token limits, which is critical for processing with token-limited models like language models.                                                                                                                                                                                         | [ 0.03108987 -0.00656008  0.01183818 ... -0.02150798 -0.00361152        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.06106624]                                                           |
| openaivec.util           | TextChunker                   | class               | Utility class for splitting text into chunks bounded by a maximum token count, using token-aware sentence segmentation based on specified separators.                                                                                                                                                                                                                                            | What is the role of the enc attribute?                                                   | It is a tiktoken.Encoding instance used to encode text and count tokens accurately.                                                                                                                                                                                                                                | [ 0.03421448  0.04242079  0.04565162 ... -0.0221958  -0.01954652        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.01741417]                                                           |
| openaivec.embeddings     | VectorizedEmbeddings          | class               | An abstract base class defining the interface for a sentence-embedding backend. It declares an abstract method `create` which must be implemented by subclasses to embed a list of input sentences into numerical vectors.                                                                                                                                                                       | What is the purpose of the VectorizedEmbeddings class?                                   | It serves as an abstract interface that defines the contract for any sentence embedding backend, ensuring that subclasses implement the `create` method to embed sentences.                                                                                                                                        | [-0.00587445  0.03139489 -0.01579419 ...  0.02471924 -0.04024738        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.00838689]                                                           |
| openaivec.embeddings     | VectorizedEmbeddings          | class               | An abstract base class defining the interface for a sentence-embedding backend. It declares an abstract method `create` which must be implemented by subclasses to embed a list of input sentences into numerical vectors.                                                                                                                                                                       | What does the create method do?                                                          | The `create` method takes a list of input strings and a batch size, and returns a list of numpy arrays representing the embeddings of each input sentence. It is abstract and must be implemented by subclasses.                                                                                                   | [ 0.01733918  0.0160381   0.0194776  ... -0.03395697 -0.02510703        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.00257318]                                                           |
| openaivec.embeddings     | VectorizedEmbeddings          | class               | An abstract base class defining the interface for a sentence-embedding backend. It declares an abstract method `create` which must be implemented by subclasses to embed a list of input sentences into numerical vectors.                                                                                                                                                                       | Are duplicates allowed in the input sentences?                                           | Yes, duplicates are allowed in the input list. The implementation may choose to handle deduplication internally.                                                                                                                                                                                                   | [ 0.06077593 -0.01524729  0.01062912 ... -0.00480143  0.04185009        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.02820216]                                                           |
| openaivec.embeddings     | VectorizedEmbeddingsOpenAI    | class               | A concrete implementation of VectorizedEmbeddings that wraps the OpenAI embeddings endpoint. It uses an OpenAI client to generate embeddings for input sentences, supports both sequential and parallel execution, and applies an exponential backoff strategy to handle rate limiting errors from OpenAI.                                                                                       | What attributes does VectorizedEmbeddingsOpenAI have?                                    | It has three attributes: `client` (an OpenAI client instance), `model_name` (the identifier of the embedding model to use), and `is_parallel` (a boolean indicating whether to run embedding requests in parallel).                                                                                                | [-0.00369965  0.02942338  0.0152207  ...  0.02052188 -0.03049107        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.00258231]                                                           |
| openaivec.embeddings     | VectorizedEmbeddingsOpenAI    | class               | A concrete implementation of VectorizedEmbeddings that wraps the OpenAI embeddings endpoint. It uses an OpenAI client to generate embeddings for input sentences, supports both sequential and parallel execution, and applies an exponential backoff strategy to handle rate limiting errors from OpenAI.                                                                                       | How does the _embed_chunk method work?                                                   | The `_embed_chunk` method sends a batch of input sentences to the OpenAI embeddings endpoint using the configured client and model. It returns a list of numpy float32 arrays representing the embeddings. It is decorated to automatically retry with exponential backoff if a RateLimitError occurs.             | [-0.01645049  0.00669724 -0.0173738  ... -0.0383368  -0.02513739        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.01101468]                                                           |
| openaivec.embeddings     | VectorizedEmbeddingsOpenAI    | class               | A concrete implementation of VectorizedEmbeddings that wraps the OpenAI embeddings endpoint. It uses an OpenAI client to generate embeddings for input sentences, supports both sequential and parallel execution, and applies an exponential backoff strategy to handle rate limiting errors from OpenAI.                                                                                       | What is the purpose of the create method in this class?                                  | The `create` method implements the abstract method from the base class. It takes input sentences and a batch size, then delegates the embedding work to either a sequential or parallel minibatch mapping function depending on the `is_parallel` flag. It returns the embeddings in the same order as the inputs. | [ 0.0483719   0.03114421  0.00020095 ...  0.00589353 -0.01775556        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.02103074]                                                           |
| openaivec.embeddings     | VectorizedEmbeddingsOpenAI    | class               | A concrete implementation of VectorizedEmbeddings that wraps the OpenAI embeddings endpoint. It uses an OpenAI client to generate embeddings for input sentences, supports both sequential and parallel execution, and applies an exponential backoff strategy to handle rate limiting errors from OpenAI.                                                                                       | How does the class handle OpenAI rate limits?                                            | The `_embed_chunk` method is decorated with a backoff decorator that catches `RateLimitError` exceptions and retries the request with exponential backoff, scaling the wait time and retrying up to 16 times.                                                                                                      | [-0.00042407  0.01931046  0.0818117  ...  0.00656221 -0.00879838        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.00104058]                                                           |
| openaivec.embeddings     | VectorizedEmbeddingsOpenAI    | class               | A concrete implementation of VectorizedEmbeddings that wraps the OpenAI embeddings endpoint. It uses an OpenAI client to generate embeddings for input sentences, supports both sequential and parallel execution, and applies an exponential backoff strategy to handle rate limiting errors from OpenAI.                                                                                       | What are map_unique_minibatch and map_unique_minibatch_parallel used for?                | These utility functions handle batching and deduplication of input sentences, and apply the embedding function (`_embed_chunk`) either sequentially or in parallel across multiple processes, respectively.                                                                                                        | [-0.0298769  -0.02788353  0.05372621 ... -0.01347901 -0.01206111        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.0023434 ]                                                           |
| openaivec.pandas_ext     | use                           | function            | Registers a custom OpenAI-compatible client instance to be used by the pandas extension helpers. This client instance is stored globally and reused for all subsequent calls.                                                                                                                                                                                                                    | What is the purpose of the 'use' function?                                               | The 'use' function allows the user to register a pre-configured OpenAI or AzureOpenAI client instance, which will then be used by all pandas extension methods in this module to interact with the OpenAI API.                                                                                                     | [-0.00614469  0.03181943 -0.02008159 ...  0.03139517 -0.06346916        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.00277713]                                                           |
| openaivec.pandas_ext     | use                           | function            | Registers a custom OpenAI-compatible client instance to be used by the pandas extension helpers. This client instance is stored globally and reused for all subsequent calls.                                                                                                                                                                                                                    | What type of client can be passed to 'use'?                                              | The client must be an instance of either 'openai.OpenAI' or 'openai.AzureOpenAI', which are compatible with the OpenAI API.                                                                                                                                                                                        | [-0.02636235  0.02433772  0.02907591 ... -0.01038325 -0.05258409        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.01781392]                                                           |
| openaivec.pandas_ext     | use_openai                    | function            | Creates and registers a default OpenAI client using a provided API key. This simplifies setup by creating the client internally and storing it globally for reuse.                                                                                                                                                                                                                               | How does 'use_openai' differ from 'use'?                                                 | 'use_openai' creates a new OpenAI client internally using the provided API key and registers it, whereas 'use' requires the user to provide a pre-configured client instance.                                                                                                                                      | [-0.01247284 -0.00902874  0.03309625 ...  0.02832117 -0.03921604        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.00541656]                                                           |
| openaivec.pandas_ext     | use_openai                    | function            | Creates and registers a default OpenAI client using a provided API key. This simplifies setup by creating the client internally and storing it globally for reuse.                                                                                                                                                                                                                               | What argument does 'use_openai' require?                                                 | It requires an 'api_key' string which is used to authenticate with the OpenAI API.                                                                                                                                                                                                                                 | [ 0.0173002   0.02955223  0.01536279 ...  0.02709636 -0.01358911        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.00560073]                                                           |
| openaivec.pandas_ext     | use_azure_openai              | function            | Creates and registers an Azure OpenAI client using the provided Azure subscription key, endpoint, and API version. This allows integration with Azure's OpenAI service.                                                                                                                                                                                                                          | What parameters are needed to use 'use_azure_openai'?                                    | It requires 'api_key' (Azure subscription key), 'endpoint' (the Azure resource endpoint URL), and 'api_version' (the REST API version string).                                                                                                                                                                     | [-0.00964736 -0.00959641  0.0563431  ... -0.0166202   0.01668387        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.00182281]                                                           |
| openaivec.pandas_ext     | use_azure_openai              | function            | Creates and registers an Azure OpenAI client using the provided Azure subscription key, endpoint, and API version. This allows integration with Azure's OpenAI service.                                                                                                                                                                                                                          | What does 'use_azure_openai' do internally?                                              | It creates an 'AzureOpenAI' client instance with the provided parameters and registers it globally for use by the pandas extension.                                                                                                                                                                                | [-0.02547258 -0.0046816   0.06306756 ...  0.00023283 -0.01745989        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.01413851]                                                           |
| openaivec.pandas_ext     | responses_model               | function            | Overrides the model used for generating text responses from the OpenAI API. It also updates the token encoding used for token counting based on the new model name.                                                                                                                                                                                                                              | What is the default model name for responses?                                            | The default model name is 'gpt-4o-mini'.                                                                                                                                                                                                                                                                           | [-0.02745681  0.00565146  0.04103405 ...  0.01227861  0.01833203        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.00079833]                                                           |
| openaivec.pandas_ext     | responses_model               | function            | Overrides the model used for generating text responses from the OpenAI API. It also updates the token encoding used for token counting based on the new model name.                                                                                                                                                                                                                              | What happens if the model name is not recognized by the 'tiktoken' library?              | A warning is logged, and the token encoding falls back to the 'o200k_base' encoding.                                                                                                                                                                                                                               | [-0.02145237 -0.03000068  0.00147964 ...  0.02362397  0.00044719        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.04235243]                                                           |
| openaivec.pandas_ext     | embeddings_model              | function            | Overrides the model used for generating text embeddings from the OpenAI API.                                                                                                                                                                                                                                                                                                                     | What is the default embeddings model?                                                    | The default embeddings model is 'text-embedding-3-small'.                                                                                                                                                                                                                                                          | [-0.01520886 -0.01832679  0.04243724 ... -0.00143699 -0.01405451        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.00303463]                                                           |
| openaivec.pandas_ext     | embeddings_model              | function            | Overrides the model used for generating text embeddings from the OpenAI API.                                                                                                                                                                                                                                                                                                                     | How do you change the embeddings model?                                                  | By calling 'embeddings_model' with the desired model name string.                                                                                                                                                                                                                                                  | [-0.00491218 -0.01635941  0.0366062  ... -0.01440326 -0.01307008        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.00121403]                                                           |
| openaivec.pandas_ext     | _get_openai_client            | function            | Retrieves the globally registered OpenAI client. If none is registered, attempts to create one from environment variables for either OpenAI or Azure OpenAI. Raises an error if no credentials are found.                                                                                                                                                                                        | What environment variables does '_get_openai_client' check for OpenAI?                   | It checks for 'OPENAI_API_KEY'.                                                                                                                                                                                                                                                                                    | [-0.01382415  0.00766441  0.0820985  ... -0.00372641 -0.01129677        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.00891634]                                                           |
| openaivec.pandas_ext     | _get_openai_client            | function            | Retrieves the globally registered OpenAI client. If none is registered, attempts to create one from environment variables for either OpenAI or Azure OpenAI. Raises an error if no credentials are found.                                                                                                                                                                                        | What environment variables does it check for Azure OpenAI?                               | It checks for 'AZURE_OPENAI_API_KEY', 'AZURE_OPENAI_ENDPOINT', and 'AZURE_OPENAI_API_VERSION'.                                                                                                                                                                                                                     | [-0.02562776  0.00922034  0.08270665 ... -0.03528424  0.00394368        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.00111569]                                                           |
| openaivec.pandas_ext     | _get_openai_client            | function            | Retrieves the globally registered OpenAI client. If none is registered, attempts to create one from environment variables for either OpenAI or Azure OpenAI. Raises an error if no credentials are found.                                                                                                                                                                                        | What happens if no client is registered and no environment variables are set?            | It raises a ValueError indicating that no OpenAI API key was found and instructs the user to set the appropriate environment variables.                                                                                                                                                                            | [-0.04086812  0.03370593  0.09328815 ... -0.04032903  0.0024628         |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.01569777]                                                           |
| openaivec.pandas_ext     | _extract_value                | function            | Converts a single Series element into a homogeneous dictionary representation. Supports Pydantic BaseModel instances and dicts. Returns an empty dict for unsupported types or None.                                                                                                                                                                                                             | What types of values can '_extract_value' handle?                                        | It can handle None, Pydantic BaseModel instances, and dicts.                                                                                                                                                                                                                                                       | [-0.00044514  0.04825476  0.00654904 ...  0.01159012  0.01762196        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.02023908]                                                           |
| openaivec.pandas_ext     | _extract_value                | function            | Converts a single Series element into a homogeneous dictionary representation. Supports Pydantic BaseModel instances and dicts. Returns an empty dict for unsupported types or None.                                                                                                                                                                                                             | What does it return if the value is not a dict or BaseModel?                             | It logs a warning and returns an empty dictionary.                                                                                                                                                                                                                                                                 | [-0.03397126  0.04107209  0.01302946 ...  0.01860868 -0.00583282        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.04089175]                                                           |
| openaivec.pandas_ext     | OpenAIVecSeriesAccessor       | class               | A pandas Series accessor registered as '.ai' that provides OpenAI-related helper methods such as generating responses, computing embeddings, counting tokens, and extracting dict-like data from Series elements.                                                                                                                                                                                | How do you access the OpenAI helpers on a pandas Series?                                 | By using the '.ai' accessor, e.g., 'series.ai.responses(...)'.                                                                                                                                                                                                                                                     | [-0.01635489 -0.03776291  0.07863165 ...  0.01122781 -0.01568936        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.00983512]                                                           |
| openaivec.pandas_ext     | OpenAIVecSeriesAccessor       | class               | A pandas Series accessor registered as '.ai' that provides OpenAI-related helper methods such as generating responses, computing embeddings, counting tokens, and extracting dict-like data from Series elements.                                                                                                                                                                                | What does the 'responses' method do?                                                     | It sends each element of the Series as a prompt to an OpenAI language model with given instructions and returns a Series of responses.                                                                                                                                                                             | [-0.05706602  0.03136273  0.0671469  ...  0.01508447 -0.01179787        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.00807649]                                                           |
| openaivec.pandas_ext     | OpenAIVecSeriesAccessor       | class               | A pandas Series accessor registered as '.ai' that provides OpenAI-related helper methods such as generating responses, computing embeddings, counting tokens, and extracting dict-like data from Series elements.                                                                                                                                                                                | What is the purpose of the 'embeddings' method?                                          | It computes OpenAI embeddings for each element in the Series and returns a Series of numpy arrays representing the embeddings.                                                                                                                                                                                     | [ 0.0034841   0.00212292  0.02750191 ... -0.02128881 -0.02216314        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.00413323]                                                           |
| openaivec.pandas_ext     | OpenAIVecSeriesAccessor       | class               | A pandas Series accessor registered as '.ai' that provides OpenAI-related helper methods such as generating responses, computing embeddings, counting tokens, and extracting dict-like data from Series elements.                                                                                                                                                                                | How does 'count_tokens' work?                                                            | It uses the 'tiktoken' library to count the number of tokens in each Series element based on the current response model's encoding.                                                                                                                                                                                | [-0.01469692  0.01987267  0.01889011 ... -0.00131037 -0.00198069        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.01354137]                                                           |
| openaivec.pandas_ext     | OpenAIVecSeriesAccessor       | class               | A pandas Series accessor registered as '.ai' that provides OpenAI-related helper methods such as generating responses, computing embeddings, counting tokens, and extracting dict-like data from Series elements.                                                                                                                                                                                | What does the 'extract' method do?                                                       | It expands a Series of dicts or Pydantic models into a DataFrame with columns corresponding to the keys, optionally prefixing columns with the Series name.                                                                                                                                                        | [-0.0382226   0.01913841 -0.02255404 ...  0.00824768 -0.03952379        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.0116091 ]                                                           |
| openaivec.pandas_ext     | OpenAIVecDataFrameAccessor    | class               | A pandas DataFrame accessor registered as '.ai' that provides OpenAI-related helper methods such as extracting nested dict-like columns and generating responses from entire rows serialized as JSON.                                                                                                                                                                                            | How do you access the OpenAI helpers on a pandas DataFrame?                              | By using the '.ai' accessor, e.g., 'df.ai.responses(...)'.                                                                                                                                                                                                                                                         | [-0.01116245 -0.01680343  0.07285469 ...  0.01146123 -0.01144928        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.01146123]                                                           |
| openaivec.pandas_ext     | OpenAIVecDataFrameAccessor    | class               | A pandas DataFrame accessor registered as '.ai' that provides OpenAI-related helper methods such as extracting nested dict-like columns and generating responses from entire rows serialized as JSON.                                                                                                                                                                                            | What does the 'extract' method do?                                                       | It flattens one column of Pydantic models or dicts into top-level columns, dropping the original column.                                                                                                                                                                                                           | [-0.0382226   0.01913841 -0.02255404 ...  0.00824768 -0.03952379        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.0116091 ]                                                           |
| openaivec.pandas_ext     | OpenAIVecDataFrameAccessor    | class               | A pandas DataFrame accessor registered as '.ai' that provides OpenAI-related helper methods such as extracting nested dict-like columns and generating responses from entire rows serialized as JSON.                                                                                                                                                                                            | What does the 'responses' method do?                                                     | It serializes each row to JSON and sends it as a prompt to an OpenAI language model with given instructions, returning a Series of responses aligned with the DataFrame's index.                                                                                                                                   | [-0.05706602  0.03136273  0.0671469  ...  0.01508447 -0.01179787        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.00807649]                                                           |
| openaivec.spark          | UDFBuilder                    | class               | A builder class for creating Spark pandas-UDFs that interact with the OpenAI or Azure OpenAI APIs. It encapsulates authentication, batching, and HTTP client configuration, allowing users to easily create UDFs for generating chat completions and embeddings within Spark DataFrames. Supports both public OpenAI and Azure OpenAI endpoints, with options for parallel execution and HTTP/2. | What is the purpose of the UDFBuilder class?                                             | UDFBuilder is designed to create Spark pandas UDFs that communicate with OpenAI or Azure OpenAI APIs, enabling large-scale data processing with chat completions and embeddings directly in Spark.                                                                                                                 | [-0.02941521  0.01529391  0.05124583 ... -0.00638078 -0.04129107        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.01229376]                                                           |
| openaivec.spark          | UDFBuilder                    | class               | A builder class for creating Spark pandas-UDFs that interact with the OpenAI or Azure OpenAI APIs. It encapsulates authentication, batching, and HTTP client configuration, allowing users to easily create UDFs for generating chat completions and embeddings within Spark DataFrames. Supports both public OpenAI and Azure OpenAI endpoints, with options for parallel execution and HTTP/2. | How do you create an instance of UDFBuilder for Azure OpenAI?                            | Use the class method of_azureopenai with parameters like api_key, api_version, endpoint, model_name, and optional batch_size, http2, ssl_verify, and is_parallel flags.                                                                                                                                            | [-0.04126972 -0.01344186  0.06783043 ... -0.02206353 -0.0101373         |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.00198615]                                                           |
| openaivec.spark          | UDFBuilder                    | class               | A builder class for creating Spark pandas-UDFs that interact with the OpenAI or Azure OpenAI APIs. It encapsulates authentication, batching, and HTTP client configuration, allowing users to easily create UDFs for generating chat completions and embeddings within Spark DataFrames. Supports both public OpenAI and Azure OpenAI endpoints, with options for parallel execution and HTTP/2. | How do you create an instance of UDFBuilder for the public OpenAI API?                   | Use the class method of_openai with parameters api_key, model_name, and optional batch_size, http2, ssl_verify, and is_parallel flags.                                                                                                                                                                             | [-0.02861198 -0.01923347  0.05571939 ... -0.00995526 -0.02828599        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.00564842]                                                           |
| openaivec.spark          | UDFBuilder                    | class               | A builder class for creating Spark pandas-UDFs that interact with the OpenAI or Azure OpenAI APIs. It encapsulates authentication, batching, and HTTP client configuration, allowing users to easily create UDFs for generating chat completions and embeddings within Spark DataFrames. Supports both public OpenAI and Azure OpenAI endpoints, with options for parallel execution and HTTP/2. | What parameters can be configured in UDFBuilder?                                         | You can configure api_key, endpoint (for Azure), api_version (for Azure), model_name, batch_size (number of rows per request), is_parallel (whether to execute vectorized requests concurrently), http2 (enable HTTP/2), and ssl_verify (TLS certificate verification).                                            | [-0.0619349   0.00558435  0.09106294 ... -0.04784962 -0.00114333        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.00317801]                                                           |
| openaivec.spark          | UDFBuilder                    | class               | A builder class for creating Spark pandas-UDFs that interact with the OpenAI or Azure OpenAI APIs. It encapsulates authentication, batching, and HTTP client configuration, allowing users to easily create UDFs for generating chat completions and embeddings within Spark DataFrames. Supports both public OpenAI and Azure OpenAI endpoints, with options for parallel execution and HTTP/2. | What does the responses method do?                                                       | It returns a pandas UDF that produces chat completions based on given system instructions. The output schema can be a string or a Pydantic model describing the expected JSON response structure.                                                                                                                  | [-0.03917537  0.02508492  0.06850404 ...  0.00845729 -0.01349391        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.00346598]                                                           |
| openaivec.spark          | UDFBuilder                    | class               | A builder class for creating Spark pandas-UDFs that interact with the OpenAI or Azure OpenAI APIs. It encapsulates authentication, batching, and HTTP client configuration, allowing users to easily create UDFs for generating chat completions and embeddings within Spark DataFrames. Supports both public OpenAI and Azure OpenAI endpoints, with options for parallel execution and HTTP/2. | What does the embeddings method do?                                                      | It returns a pandas UDF that generates embedding vectors for input strings, outputting an array of floats representing the embeddings.                                                                                                                                                                             | [-0.01763217 -0.02698354  0.03459362 ... -0.01859955 -0.02092127        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.00695226]                                                           |
| openaivec.prompt         | Example                       | class               | Represents a single input/output example used in few‑shot prompts. It holds the input text and the expected output text that demonstrate the desired behavior for the model.                                                                                                                                                                                                                     | What attributes does the Example class have?                                             | The Example class has two attributes: 'input' (a string representing the input text) and 'output' (a string representing the expected output).                                                                                                                                                                     | [-0.02702614  0.00234793  0.01094669 ... -0.00840095 -0.01482034        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.01299016]                                                           |
| openaivec.prompt         | Example                       | class               | Represents a single input/output example used in few‑shot prompts. It holds the input text and the expected output text that demonstrate the desired behavior for the model.                                                                                                                                                                                                                     | What is the purpose of the Example class?                                                | It encapsulates a single example pair of input and output to be used in few-shot learning prompts, helping the model understand the expected behavior.                                                                                                                                                             | [-0.01215695 -0.00844365 -0.02812276 ...  0.0002933  -0.03882578        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.00937198]                                                           |
| openaivec.prompt         | FewShotPrompt                 | class               | Represents a complete few-shot prompt definition, including the purpose of the prompt, a list of cautionary notes, and a list of input/output examples. This model is used to generate the XML prompt sent to the language model.                                                                                                                                                                | What fields are included in FewShotPrompt?                                               | It includes 'purpose' (a string describing the prompt's goal), 'cautions' (a list of strings with warnings or edge cases), and 'examples' (a list of Example instances).                                                                                                                                           | [-0.00160104  0.00097476 -0.0050713  ... -0.02967625 -0.01111896        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.01911613]                                                           |
| openaivec.prompt         | FewShotPrompt                 | class               | Represents a complete few-shot prompt definition, including the purpose of the prompt, a list of cautionary notes, and a list of input/output examples. This model is used to generate the XML prompt sent to the language model.                                                                                                                                                                | How is FewShotPrompt used?                                                               | It serves as the structured data model for the prompt content that will be rendered into XML and provided to the LLM as part of the system prompt.                                                                                                                                                                 | [-0.0011549  -0.00258811 -0.02538448 ... -0.03916996 -0.04234302        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.00859926]                                                           |
| openaivec.prompt         | Step                          | class               | Represents a single refinement iteration of the prompt produced by the LLM. Each step includes an identifier, an analysis explaining the change, and the updated prompt after the modification.                                                                                                                                                                                                  | What information does a Step contain?                                                    | A Step contains an 'id' (integer iteration number), 'analysis' (a natural language explanation of the change), and 'prompt' (the FewShotPrompt after the change).                                                                                                                                                  | [-0.0091029   0.02098002 -0.01319199 ... -0.019853   -0.01894271        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.02673075]                                                           |
| openaivec.prompt         | Step                          | class               | Represents a single refinement iteration of the prompt produced by the LLM. Each step includes an identifier, an analysis explaining the change, and the updated prompt after the modification.                                                                                                                                                                                                  | What is the role of Step in prompt refinement?                                           | It tracks each iteration of prompt improvement, allowing inspection of how the prompt evolves and why changes were made.                                                                                                                                                                                           | [ 0.01545758  0.0365435   0.00675674 ... -0.03453144 -0.03605408        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.00868044]                                                           |
| openaivec.prompt         | Request                       | class               | A simple wrapper model containing a FewShotPrompt instance, used to structure the request sent to the LLM for prompt improvement.                                                                                                                                                                                                                                                                | What does the Request class encapsulate?                                                 | It encapsulates a single attribute 'prompt' which is a FewShotPrompt instance.                                                                                                                                                                                                                                     | [-0.02573247  0.01712496  0.00433592 ...  0.00169191 -0.00189777        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.00206021]                                                           |
| openaivec.prompt         | Response                      | class               | Represents the response from the LLM containing a list of Step iterations that show the progressive refinements made to the prompt.                                                                                                                                                                                                                                                              | What does the Response class contain?                                                    | It contains 'iterations', a list of Step instances representing each refinement iteration returned by the LLM.                                                                                                                                                                                                     | [-0.06169859  0.05038077  0.02898131 ...  0.01469668 -0.00161646        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.02377566]                                                           |
| openaivec.prompt         | _render_prompt                | function            | Converts a FewShotPrompt instance into its XML string representation. It creates XML elements for purpose, cautions, and examples, formatting them according to the expected schema.                                                                                                                                                                                                             | What is the output format of _render_prompt?                                             | It outputs a string containing the XML representation of the FewShotPrompt.                                                                                                                                                                                                                                        | [ 0.00560759  0.01683227  0.0164903  ...  0.01754153 -0.01272236        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.01731355]                                                           |
| openaivec.prompt         | _render_prompt                | function            | Converts a FewShotPrompt instance into its XML string representation. It creates XML elements for purpose, cautions, and examples, formatting them according to the expected schema.                                                                                                                                                                                                             | How does _render_prompt handle empty cautions?                                           | It always outputs the <Cautions> element, but if there are no cautions, it will be empty.                                                                                                                                                                                                                          | [ 0.01930845  0.01242875  0.04759166 ...  0.01128214 -0.02574932        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.00594896]                                                           |
| openaivec.prompt         | FewShotPromptBuilder          | class               | A builder class to construct and iteratively improve few-shot prompts. It allows setting the purpose, adding cautions and examples, and refining the prompt by calling an LLM to generate improvement steps. It supports outputting the prompt as XML or JSON.                                                                                                                                   | How do you set the purpose of the prompt using FewShotPromptBuilder?                     | By calling the 'purpose' method with a string describing the prompt's goal.                                                                                                                                                                                                                                        | [ 0.0179185  -0.02749075 -0.00111328 ... -0.02671271 -0.04559787        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.03201753]                                                           |
| openaivec.prompt         | FewShotPromptBuilder          | class               | A builder class to construct and iteratively improve few-shot prompts. It allows setting the purpose, adding cautions and examples, and refining the prompt by calling an LLM to generate improvement steps. It supports outputting the prompt as XML or JSON.                                                                                                                                   | How can cautions be added?                                                               | By calling the 'caution' method with a caution string; multiple calls append multiple cautions.                                                                                                                                                                                                                    | [ 0.04664813  0.02847069  0.03591686 ... -0.00983177  0.00182439        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.02083679]                                                           |
| openaivec.prompt         | FewShotPromptBuilder          | class               | A builder class to construct and iteratively improve few-shot prompts. It allows setting the purpose, adding cautions and examples, and refining the prompt by calling an LLM to generate improvement steps. It supports outputting the prompt as XML or JSON.                                                                                                                                   | How are examples added?                                                                  | Using the 'example' method, passing input and output values as strings or Pydantic models.                                                                                                                                                                                                                         | [ 0.00362261 -0.01318793  0.01794054 ... -0.01417256 -0.00773754        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.01504865]                                                           |
| openaivec.prompt         | FewShotPromptBuilder          | class               | A builder class to construct and iteratively improve few-shot prompts. It allows setting the purpose, adding cautions and examples, and refining the prompt by calling an LLM to generate improvement steps. It supports outputting the prompt as XML or JSON.                                                                                                                                   | What does the 'improve' method do?                                                       | It sends the current prompt to an LLM to iteratively refine it, receiving multiple improvement steps and updating the internal prompt accordingly. It requires at least 5 examples to run.                                                                                                                         | [-0.03149921  0.03178376  0.01076294 ... -0.01180865 -0.04897032        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.01254847]                                                           |
| openaivec.prompt         | FewShotPromptBuilder          | class               | A builder class to construct and iteratively improve few-shot prompts. It allows setting the purpose, adding cautions and examples, and refining the prompt by calling an LLM to generate improvement steps. It supports outputting the prompt as XML or JSON.                                                                                                                                   | How can the prompt be output as XML?                                                     | By calling the 'build' or 'build_xml' methods, which validate and then render the prompt to XML.                                                                                                                                                                                                                   | [ 0.02269589  0.03046916  0.032159   ... -0.02422974 -0.01366173        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.02737545]                                                           |
| openaivec.prompt         | FewShotPromptBuilder          | class               | A builder class to construct and iteratively improve few-shot prompts. It allows setting the purpose, adding cautions and examples, and refining the prompt by calling an LLM to generate improvement steps. It supports outputting the prompt as XML or JSON.                                                                                                                                   | What is the purpose of the 'explain' method?                                             | It prints a diff of each improvement iteration, showing changes between prompt versions along with the LLM's analysis.                                                                                                                                                                                             | [-0.00272845  0.04553072 -0.02156933 ...  0.01094096 -0.02538847        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.01075748]                                                           |
| openaivec.prompt         | FewShotPromptBuilder          | class               | A builder class to construct and iteratively improve few-shot prompts. It allows setting the purpose, adding cautions and examples, and refining the prompt by calling an LLM to generate improvement steps. It supports outputting the prompt as XML or JSON.                                                                                                                                   | How does the builder ensure the prompt is valid before output?                           | It validates that the purpose is set and that there is at least one example, raising ValueError if not.                                                                                                                                                                                                            | [ 0.02572622  0.01313475  0.02801122 ... -0.02305772 -0.03173433        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.02657311]                                                           |
| openaivec.beta.batch     | JsonlUDFBuilder               | class               | A frozen dataclass that encapsulates the building of a PySpark pandas UDF to convert input data into JSON Lines formatted strings for chat completion requests to a specified model.                                                                                                                                                                                                             | What is the purpose of the JsonlUDFBuilder class?                                        | The JsonlUDFBuilder class is designed to create a PySpark pandas UDF that transforms input data (custom IDs and user messages) into JSON Lines format strings. These strings represent HTTP POST requests to a chat completion API using a specified model.                                                        | [-0.02769756 -0.02106194  0.01810049 ... -0.02179924 -0.02528908        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.00494292]                                                           |
| openaivec.beta.batch     | JsonlUDFBuilder               | class               | A frozen dataclass that encapsulates the building of a PySpark pandas UDF to convert input data into JSON Lines formatted strings for chat completion requests to a specified model.                                                                                                                                                                                                             | What does the 'model_name' attribute represent?                                          | The 'model_name' attribute stores the name of the model to be used in the chat completion requests within the generated JSON payloads.                                                                                                                                                                             | [0.00452573 0.03461436 0.00183018 ... 0.01953523 0.02018508 0.00623324] |
| openaivec.beta.batch     | JsonlUDFBuilder               | class               | A frozen dataclass that encapsulates the building of a PySpark pandas UDF to convert input data into JSON Lines formatted strings for chat completion requests to a specified model.                                                                                                                                                                                                             | What does the 'to_jsonl' method do?                                                      | The 'to_jsonl' method returns a PySpark pandas UDF that takes iterators of pandas Series (custom IDs and user messages) and yields pandas Series of JSON strings. Each JSON string encodes a POST request with the system message, user message, model name, and parameters like temperature and top_p.            | [-0.00423869 -0.03800544 -0.00452106 ...  0.01684304 -0.02308625        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.00510442]                                                           |
| openaivec.beta.batch     | JsonlUDFBuilder               | class               | A frozen dataclass that encapsulates the building of a PySpark pandas UDF to convert input data into JSON Lines formatted strings for chat completion requests to a specified model.                                                                                                                                                                                                             | How does the returned UDF process the input data?                                        | The UDF iterates over pairs of custom_id and user_message Series, constructs a pandas DataFrame from them, and applies a lambda function row-wise to generate JSON strings representing the API request. It then yields these JSON strings as a pandas Series.                                                     | [-0.03840687  0.04318549  0.06532472 ... -0.014679   -0.01409438        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.02244425]                                                           |
| openaivec.beta.batch     | JsonlUDFBuilder               | class               | A frozen dataclass that encapsulates the building of a PySpark pandas UDF to convert input data into JSON Lines formatted strings for chat completion requests to a specified model.                                                                                                                                                                                                             | What are the roles of 'temperature' and 'top_p' parameters?                              | The 'temperature' and 'top_p' parameters control the randomness and diversity of the model's output in the chat completion request. They are included in the JSON payload sent to the API.                                                                                                                         | [ 0.00102343  0.00301845  0.01945186 ... -0.03023466  0.06983405        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.02397368]                                                           |
| openaivec.beta.batch     | JsonlUDFBuilder               | class               | A frozen dataclass that encapsulates the building of a PySpark pandas UDF to convert input data into JSON Lines formatted strings for chat completion requests to a specified model.                                                                                                                                                                                                             | Why is the class marked as frozen?                                                       | Marking the class as frozen makes its instances immutable, ensuring that the 'model_name' attribute cannot be changed after instantiation, which can help maintain consistency and thread safety.                                                                                                                  | [-0.01467541  0.033842    0.01314145 ...  0.01435913  0.02395824        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.00078131]                                                           |
| openaivec.beta.schema    | Entity                        | class               | Represents an entity extracted from text, including its textual content and its classification according to schema.org hierarchy.                                                                                                                                                                                                                                                                | What attributes does the Entity class have?                                              | The Entity class has two attributes: 'body', a string representing the text of the entity, and 'entity_class', a string representing the full schema.org class hierarchy of the entity (e.g., 'Thing.Person').                                                                                                     | [-0.03689022  0.03209228  0.01593599 ... -0.01883678 -0.00939391        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.00484995]                                                           |
| openaivec.beta.schema    | Entity                        | class               | Represents an entity extracted from text, including its textual content and its classification according to schema.org hierarchy.                                                                                                                                                                                                                                                                | What is the purpose of the entity_class attribute?                                       | The 'entity_class' attribute specifies the schema.org classification of the entity, including the full hierarchy from 'Thing' down to the most specific class, using dot-separated notation.                                                                                                                       | [-0.00746422  0.01220313 -0.01919893 ...  0.01130888 -0.02922423        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.01350495]                                                           |
| openaivec.beta.schema    | Relationship                  | class               | Models a relationship between two entities, specifying the tail entity, the head entity, and the property that defines their relationship according to schema.org.                                                                                                                                                                                                                               | What are the components of the Relationship class?                                       | The Relationship class contains three attributes: 'tail' (an Entity instance representing the source entity), 'head' (an Entity instance representing the target entity), and 'property' (a string indicating the schema.org property that relates the two entities).                                              | [-0.04247501  0.01406106  0.02601296 ...  0.03372001  0.01314576        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.00924581]                                                           |
| openaivec.beta.schema    | Relationship                  | class               | Models a relationship between two entities, specifying the tail entity, the head entity, and the property that defines their relationship according to schema.org.                                                                                                                                                                                                                               | How are entities linked in the Relationship class?                                       | Entities are linked by the 'property' attribute, which names the schema.org property that defines the relationship from the tail entity to the head entity.                                                                                                                                                        | [-0.04330662 -0.01998859  0.013841   ...  0.00611192 -0.01165902        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.01533925]                                                           |
| openaivec.beta.schema    | RelationshipResponse          | class               | Encapsulates a response containing a list of relationships extracted from text, each represented as a Relationship object.                                                                                                                                                                                                                                                                       | What does the RelationshipResponse class represent?                                      | It represents the output structure containing a list of extracted relationships, where each relationship details two entities and their schema.org property connection.                                                                                                                                            | [-0.03991058  0.02168166  0.00220215 ...  0.02998731  0.01772595        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.00914164]                                                           |
| openaivec.beta.schema    | RelationshipResponse          | class               | Encapsulates a response containing a list of relationships extracted from text, each represented as a Relationship object.                                                                                                                                                                                                                                                                       | What type of data does the 'relationships' attribute hold?                               | The 'relationships' attribute is a list of Relationship objects.                                                                                                                                                                                                                                                   | [-0.00263802  0.01961179  0.04463829 ... -0.00321085  0.01904391        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.0200212 ]                                                           |
| openaivec.beta.schema    | schemaorg_extraction_prompt   | variable            | A detailed prompt string designed to instruct a text analysis system to extract entities and their relationships from input text according to schema.org standards, specifying output format and constraints.                                                                                                                                                                                    | What is the purpose of the schemaorg_extraction_prompt variable?                         | It provides detailed instructions and an example for extracting entities and relationships from text, ensuring compliance with schema.org classes and properties, and specifying the exact JSON output format.                                                                                                     | [ 0.04619913  0.00194218  0.03643346 ... -0.00871398 -0.04074037        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.00304238]                                                           |
| openaivec.beta.schema    | schemaorg_extraction_prompt   | variable            | A detailed prompt string designed to instruct a text analysis system to extract entities and their relationships from input text according to schema.org standards, specifying output format and constraints.                                                                                                                                                                                    | What key instructions does the prompt include?                                           | The prompt instructs to identify all entities, classify them with full schema.org hierarchy, find relationships using only schema.org properties, avoid inventing classes or properties, and return results in a specified JSON structure with no extra text.                                                      | [ 0.0442342   0.01671922  0.02590982 ... -0.02903491  0.00979431        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.00957413]                                                           |
| openaivec.beta.schema    | schemaorg_extraction_prompt   | variable            | A detailed prompt string designed to instruct a text analysis system to extract entities and their relationships from input text according to schema.org standards, specifying output format and constraints.                                                                                                                                                                                    | What example is provided in the prompt?                                                  | An example input text about 'The Oscars broadcast' and 'Colosseum' is given, along with the expected JSON output showing two relationships with their entities and properties classified according to schema.org.                                                                                                  | [ 0.0285543  -0.01285379 -0.0196801  ... -0.02043535 -0.00268332        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.00201703]                                                           |
| openaivec.aio.responses  | AsyncBatchResponses           | class               | A stateless asynchronous wrapper that converts OpenAI's JSON-mode API into a batched API. It allows submitting multiple user prompts in a single JSON request and receiving responses asynchronously in the original order. It also manages concurrency limits for requests to the OpenAI API.                                                                                                   | What is the purpose of the AsyncBatchResponses class?                                    | It provides an asynchronous interface to batch multiple prompts into a single JSON-mode request to the OpenAI API, returning responses in the original order while controlling concurrency.                                                                                                                        | [-0.02701208 -0.00284884  0.01162987 ...  0.00580591 -0.04370521        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.00695747]                                                           |
| openaivec.aio.responses  | AsyncBatchResponses           | class               | A stateless asynchronous wrapper that converts OpenAI's JSON-mode API into a batched API. It allows submitting multiple user prompts in a single JSON request and receiving responses asynchronously in the original order. It also manages concurrency limits for requests to the OpenAI API.                                                                                                   | What are the main attributes of AsyncBatchResponses?                                     | Attributes include the OpenAI async client, model name, system message, temperature, top_p, response format type, max concurrency limit, and internal fields for vectorized system message, model schema, and a semaphore for concurrency control.                                                                 | [-0.03049028  0.0022402   0.01776413 ... -0.01844636 -0.02316947        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.00206636]                                                           |
| openaivec.aio.responses  | AsyncBatchResponses           | class               | A stateless asynchronous wrapper that converts OpenAI's JSON-mode API into a batched API. It allows submitting multiple user prompts in a single JSON request and receiving responses asynchronously in the original order. It also manages concurrency limits for requests to the OpenAI API.                                                                                                   | How does AsyncBatchResponses handle concurrency?                                         | It uses an asyncio.Semaphore initialized with max_concurrency to limit the number of concurrent requests to the OpenAI API.                                                                                                                                                                                        | [-0.02668401 -0.04087333  0.05086403 ... -0.01493546 -0.01294997        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.0020756 ]                                                           |
| openaivec.aio.responses  | AsyncBatchResponses           | class               | A stateless asynchronous wrapper that converts OpenAI's JSON-mode API into a batched API. It allows submitting multiple user prompts in a single JSON request and receiving responses asynchronously in the original order. It also manages concurrency limits for requests to the OpenAI API.                                                                                                   | What is the role of the _vectorized_system_message attribute?                            | It stores a processed version of the system message, prepared by the _vectorize_system_message function, to be used as instructions in API calls.                                                                                                                                                                  | [ 0.02654271  0.04940431 -0.05784594 ...  0.00851774 -0.00270893        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.03952345]                                                           |
| openaivec.aio.responses  | __post_init__                 | function            | Post-initialization method for the dataclass that sets up the vectorized system message and initializes the concurrency semaphore. Uses object.__setattr__ because the dataclass is frozen.                                                                                                                                                                                                      | Why is object.__setattr__ used in __post_init__?                                         | Because the dataclass is frozen (immutable), direct attribute assignment is not allowed, so object.__setattr__ is used to bypass immutability for initialization.                                                                                                                                                  | [ 0.02520751  0.03326306 -0.03344408 ...  0.00199267  0.02443817        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.03360248]                                                           |
| openaivec.aio.responses  | __post_init__                 | function            | Post-initialization method for the dataclass that sets up the vectorized system message and initializes the concurrency semaphore. Uses object.__setattr__ because the dataclass is frozen.                                                                                                                                                                                                      | What does __post_init__ initialize?                                                      | It initializes the _vectorized_system_message by processing the system_message and creates an asyncio.Semaphore with max_concurrency to control concurrent API calls.                                                                                                                                              | [-0.02712648  0.00428867  0.00263766 ... -0.00097761  0.00428209        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.01226743]                                                           |
| openaivec.aio.responses  | _request_llm                  | function            | Makes a single asynchronous call to the OpenAI JSON-mode endpoint with a batch of user messages, respecting concurrency limits and handling rate limiting with exponential backoff. Returns parsed responses containing assistant messages.                                                                                                                                                      | What does _request_llm do?                                                               | It sends a batch of user messages to the OpenAI API asynchronously, using a semaphore to limit concurrency, and returns the parsed response containing assistant messages.                                                                                                                                         | [-0.0135437   0.00213964  0.02233754 ... -0.00586379  0.0026194         |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.03464598]                                                           |
| openaivec.aio.responses  | _request_llm                  | function            | Makes a single asynchronous call to the OpenAI JSON-mode endpoint with a batch of user messages, respecting concurrency limits and handling rate limiting with exponential backoff. Returns parsed responses containing assistant messages.                                                                                                                                                      | How does _request_llm handle rate limiting?                                              | It is decorated with a backoff decorator that retries on RateLimitError with exponential backoff, up to a maximum number of retries.                                                                                                                                                                               | [ 0.00343399 -0.01546495  0.03515635 ... -0.01816823  0.01575312        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.02489212]                                                           |
| openaivec.aio.responses  | _request_llm                  | function            | Makes a single asynchronous call to the OpenAI JSON-mode endpoint with a batch of user messages, respecting concurrency limits and handling rate limiting with exponential backoff. Returns parsed responses containing assistant messages.                                                                                                                                                      | What is the purpose of the nested MessageT and ResponseT classes?                        | They define Pydantic models for the expected response format, where MessageT represents an assistant message with an id and body, and ResponseT wraps a list of such messages.                                                                                                                                     | [-0.0330671   0.01225973  0.03287172 ...  0.01230247 -0.00311836        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.01484844]                                                           |
| openaivec.aio.responses  | _request_llm                  | function            | Makes a single asynchronous call to the OpenAI JSON-mode endpoint with a batch of user messages, respecting concurrency limits and handling rate limiting with exponential backoff. Returns parsed responses containing assistant messages.                                                                                                                                                      | How does _request_llm ensure concurrency limits?                                         | It uses an async with block to acquire the semaphore before making the API call, ensuring no more than max_concurrency calls run simultaneously.                                                                                                                                                                   | [ 0.01823352 -0.01332664  0.0653507  ... -0.02008753  0.00623466        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.02535685]                                                           |
| openaivec.aio.responses  | _predict_chunk                | function            | Helper asynchronous method that processes a minibatch of user prompt strings. It converts strings to Message objects with stable indices, calls _request_llm to get responses, and reorders the responses to match the original input order. The method is pure and side-effect free.                                                                                                            | What is the input and output of _predict_chunk?                                          | Input is a list of user prompt strings; output is a list of responses of type T, ordered to correspond with the input prompts.                                                                                                                                                                                     | [ 0.03300594 -0.02030791 -0.00916646 ... -0.02577543 -0.00440191        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.0016277 ]                                                           |
| openaivec.aio.responses  | _predict_chunk                | function            | Helper asynchronous method that processes a minibatch of user prompt strings. It converts strings to Message objects with stable indices, calls _request_llm to get responses, and reorders the responses to match the original input order. The method is pure and side-effect free.                                                                                                            | How does _predict_chunk maintain the order of responses?                                 | It assigns each input string an id, sends them as Message objects, then reconstructs the output list by matching response ids to input ids.                                                                                                                                                                        | [ 0.00508926 -0.02836204  0.00345248 ... -0.01884618  0.01084119        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.01391596]                                                           |
| openaivec.aio.responses  | _predict_chunk                | function            | Helper asynchronous method that processes a minibatch of user prompt strings. It converts strings to Message objects with stable indices, calls _request_llm to get responses, and reorders the responses to match the original input order. The method is pure and side-effect free.                                                                                                            | Is _predict_chunk stateful or pure?                                                      | It is pure; it has no side effects and its output depends only on its input arguments.                                                                                                                                                                                                                             | [ 0.0138341  -0.02425221 -0.0378769  ... -0.03960453 -0.00261271        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.00130799]                                                           |
| openaivec.aio.responses  | parse                         | function            | Public asynchronous method to process a list of input prompts in batches. It de-duplicates inputs internally, splits them into batches of a specified size, and calls _predict_chunk on each batch. Returns a list of responses in the same order as the original inputs.                                                                                                                        | What does the parse method do?                                                           | It asynchronously processes multiple prompts in batches, returning the assistant responses in the same order as the inputs.                                                                                                                                                                                        | [-0.02220551  0.00792778 -0.00546444 ...  0.00282238 -0.02663631        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.00561578]                                                           |
| openaivec.aio.responses  | parse                         | function            | Public asynchronous method to process a list of input prompts in batches. It de-duplicates inputs internally, splits them into batches of a specified size, and calls _predict_chunk on each batch. Returns a list of responses in the same order as the original inputs.                                                                                                                        | How does parse handle batching?                                                          | It uses the utility function map to apply _predict_chunk to batches of inputs, with batch size controlled by the batch_size argument.                                                                                                                                                                              | [-0.02723623 -0.04020714  0.01937548 ... -0.01488554 -0.01619342        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.01593724]                                                           |
| openaivec.aio.responses  | parse                         | function            | Public asynchronous method to process a list of input prompts in batches. It de-duplicates inputs internally, splits them into batches of a specified size, and calls _predict_chunk on each batch. Returns a list of responses in the same order as the original inputs.                                                                                                                        | Does parse handle duplicate inputs?                                                      | Yes, duplicate entries are de-duplicated internally to save tokens during API calls.                                                                                                                                                                                                                               | [ 0.03706963 -0.00873534  0.00649594 ... -0.00210133  0.01190081        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.01720128]                                                           |
| openaivec.aio.util       | map                           | function            | Asynchronously maps an asynchronous function `f` over a list of inputs in batches, preserving the order of the original inputs. It handles duplicate inputs by processing only unique inputs and then reconstructing the full output list accordingly.                                                                                                                                           | What is the purpose of the `map` function?                                               | The `map` function applies an asynchronous function `f` to a list of inputs in batches, allowing concurrent processing to improve efficiency. It returns the results in the same order as the original inputs.                                                                                                     | [ 0.01265605  0.04409528  0.0378677  ...  0.01397439 -0.03297102        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.00533613]                                                           |
| openaivec.aio.util       | map                           | function            | Asynchronously maps an asynchronous function `f` over a list of inputs in batches, preserving the order of the original inputs. It handles duplicate inputs by processing only unique inputs and then reconstructing the full output list accordingly.                                                                                                                                           | How does the function handle duplicate inputs?                                           | The function first computes hashes of the string representations of inputs to identify unique inputs. It processes only these unique inputs in batches, then reconstructs the output list by mapping back to the original input order, including duplicates.                                                       | [ 0.04745094  0.0010995   0.03010421 ... -0.00735288  0.00081628        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.01499438]                                                           |
| openaivec.aio.util       | map                           | function            | Asynchronously maps an asynchronous function `f` over a list of inputs in batches, preserving the order of the original inputs. It handles duplicate inputs by processing only unique inputs and then reconstructing the full output list accordingly.                                                                                                                                           | Why are inputs hashed using `hash(str(v))` instead of just `hash(v)`?                    | Using `hash(str(v))` ensures that inputs which may not be directly hashable (like lists or dicts) can still be hashed by converting them to strings first. This allows the function to handle a wider variety of input types.                                                                                      | [-0.00756841  0.02548524  0.00945768 ... -0.03662794 -0.02086703        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.00134319]                                                           |
| openaivec.aio.util       | map                           | function            | Asynchronously maps an asynchronous function `f` over a list of inputs in batches, preserving the order of the original inputs. It handles duplicate inputs by processing only unique inputs and then reconstructing the full output list accordingly.                                                                                                                                           | What is the role of the `batch_size` parameter?                                          | `batch_size` determines the size of each batch of inputs that the asynchronous function `f` will process at once. This controls concurrency and resource usage during processing.                                                                                                                                  | [ 0.01663812 -0.04217083  0.04499618 ...  0.00674943  0.01385464        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.00794235]                                                           |
| openaivec.aio.util       | map                           | function            | Asynchronously maps an asynchronous function `f` over a list of inputs in batches, preserving the order of the original inputs. It handles duplicate inputs by processing only unique inputs and then reconstructing the full output list accordingly.                                                                                                                                           | How does the function ensure the output order matches the input order?                   | After processing unique inputs and obtaining their outputs, the function uses the original hashes to map each input back to its corresponding output, preserving the original input order including duplicates.                                                                                                    | [ 0.00730163 -0.00033482  0.00434681 ... -0.00168003 -0.00511732        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.0024151 ]                                                           |
| openaivec.aio.util       | map                           | function            | Asynchronously maps an asynchronous function `f` over a list of inputs in batches, preserving the order of the original inputs. It handles duplicate inputs by processing only unique inputs and then reconstructing the full output list accordingly.                                                                                                                                           | What happens if the number of unique outputs does not match the number of unique inputs? | The function raises a `ValueError` indicating a mismatch, suggesting that the asynchronous function `f` did not return the expected number of outputs for the given inputs.                                                                                                                                        | [ 0.00571908 -0.00385956  0.05168742 ...  0.00447069 -0.0206802         |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.03200071]                                                           |
| openaivec.aio.util       | map                           | function            | Asynchronously maps an asynchronous function `f` over a list of inputs in batches, preserving the order of the original inputs. It handles duplicate inputs by processing only unique inputs and then reconstructing the full output list accordingly.                                                                                                                                           | What types are used for the inputs and outputs?                                          | The function is generic, using type variables `T` for input elements and `U` for output elements. It accepts a list of `T` and returns a list of `U`.                                                                                                                                                              | [-0.00442956 -0.01444281  0.01927857 ... -0.00670559 -0.01248916        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.01310814]                                                           |
| openaivec.aio.util       | map                           | function            | Asynchronously maps an asynchronous function `f` over a list of inputs in batches, preserving the order of the original inputs. It handles duplicate inputs by processing only unique inputs and then reconstructing the full output list accordingly.                                                                                                                                           | What is the expected signature of the asynchronous function `f`?                         | `f` should be an asynchronous callable that takes a list of inputs (`List[T]`) and returns an awaitable that resolves to a list of outputs (`List[U]`) of the same length.                                                                                                                                         | [ 0.03582673  0.02277079 -0.00965702 ...  0.01504774 -0.00563487        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.03060949]                                                           |
| openaivec.aio.embeddings | AsyncBatchEmbeddings          | class               | A dataclass providing an asynchronous interface to generate embeddings using OpenAI's /embeddings endpoint. It manages concurrency with a semaphore, handles rate limits with automatic exponential backoff, and efficiently processes batches of input strings including de-duplication to avoid redundant API calls.                                                                           | What is the purpose of the AsyncBatchEmbeddings class?                                   | It provides an asynchronous wrapper around the OpenAI embeddings API, allowing users to generate embeddings for multiple input strings concurrently while managing rate limits and batching efficiently.                                                                                                           | [-0.00124631 -0.02551957  0.00486776 ... -0.00220495 -0.04848478        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.01309716]                                                           |
| openaivec.aio.embeddings | AsyncBatchEmbeddings          | class               | A dataclass providing an asynchronous interface to generate embeddings using OpenAI's /embeddings endpoint. It manages concurrency with a semaphore, handles rate limits with automatic exponential backoff, and efficiently processes batches of input strings including de-duplication to avoid redundant API calls.                                                                           | How does AsyncBatchEmbeddings handle concurrency?                                        | It uses an asyncio.Semaphore initialized with max_concurrency to limit the number of concurrent API requests.                                                                                                                                                                                                      | [ 0.00506301 -0.05116048  0.04023584 ... -0.01966679 -0.02918928        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.00139073]                                                           |
| openaivec.aio.embeddings | AsyncBatchEmbeddings          | class               | A dataclass providing an asynchronous interface to generate embeddings using OpenAI's /embeddings endpoint. It manages concurrency with a semaphore, handles rate limits with automatic exponential backoff, and efficiently processes batches of input strings including de-duplication to avoid redundant API calls.                                                                           | What is the role of the _embed_chunk method?                                             | It is a private asynchronous method that sends a batch of input strings to the OpenAI embeddings API, returning their embeddings as numpy float32 arrays. It respects concurrency limits and applies exponential backoff on rate limit errors.                                                                     | [ 0.0017356   0.0064712  -0.02349738 ... -0.03912879 -0.03073507        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.01120209]                                                           |
| openaivec.aio.embeddings | AsyncBatchEmbeddings          | class               | A dataclass providing an asynchronous interface to generate embeddings using OpenAI's /embeddings endpoint. It manages concurrency with a semaphore, handles rate limits with automatic exponential backoff, and efficiently processes batches of input strings including de-duplication to avoid redundant API calls.                                                                           | How does the class handle duplicate inputs?                                              | The public create method uses an external utility (openaivec.aio.map) that efficiently handles batching and de-duplication, ensuring duplicate inputs are embedded only once and results are reused.                                                                                                               | [ 0.04446565  0.00582853  0.05892217 ...  0.01149412  0.00773927        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.00536195]                                                           |
| openaivec.aio.embeddings | AsyncBatchEmbeddings          | class               | A dataclass providing an asynchronous interface to generate embeddings using OpenAI's /embeddings endpoint. It manages concurrency with a semaphore, handles rate limits with automatic exponential backoff, and efficiently processes batches of input strings including de-duplication to avoid redundant API calls.                                                                           | What parameters are required to instantiate AsyncBatchEmbeddings?                        | You need to provide an AsyncOpenAI client instance, the model name as a string (e.g., "text-embedding-3-small"), and optionally the max_concurrency integer to limit concurrent requests (default is 8).                                                                                                           | [-0.01613588 -0.03210114  0.02846934 ... -0.0166843  -0.01265033        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.00068667]                                                           |
| openaivec.aio.embeddings | AsyncBatchEmbeddings          | class               | A dataclass providing an asynchronous interface to generate embeddings using OpenAI's /embeddings endpoint. It manages concurrency with a semaphore, handles rate limits with automatic exponential backoff, and efficiently processes batches of input strings including de-duplication to avoid redundant API calls.                                                                           | What does the create method do?                                                          | It is the public asynchronous method to generate embeddings for a list of input strings. It batches inputs according to batch_size, handles duplicates, and returns a list of numpy float32 arrays corresponding to each input string's embedding.                                                                 | [ 0.01733918  0.0160381   0.0194776  ... -0.03395697 -0.02510703        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.00257318]                                                           |
| openaivec.aio.embeddings | AsyncBatchEmbeddings          | class               | A dataclass providing an asynchronous interface to generate embeddings using OpenAI's /embeddings endpoint. It manages concurrency with a semaphore, handles rate limits with automatic exponential backoff, and efficiently processes batches of input strings including de-duplication to avoid redundant API calls.                                                                           | How are rate limits handled in this class?                                               | The _embed_chunk method is decorated with a backoff decorator that catches RateLimitError exceptions and retries the request with exponential backoff, scaling wait times up to 60 seconds and retrying up to 16 times.                                                                                            | [-0.00292804  0.02114959  0.05034198 ... -0.00874012  0.00484396        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.00454608]                                                           |
| openaivec.aio.embeddings | AsyncBatchEmbeddings          | class               | A dataclass providing an asynchronous interface to generate embeddings using OpenAI's /embeddings endpoint. It manages concurrency with a semaphore, handles rate limits with automatic exponential backoff, and efficiently processes batches of input strings including de-duplication to avoid redundant API calls.                                                                           | What is the data type of the embeddings returned?                                        | Each embedding is a numpy ndarray with dtype float32.                                                                                                                                                                                                                                                              | [-0.01392882  0.00889062  0.02443328 ... -0.00638597 -0.00069268        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.0196091 ]                                                           |
| openaivec.aio.embeddings | AsyncBatchEmbeddings          | class               | A dataclass providing an asynchronous interface to generate embeddings using OpenAI's /embeddings endpoint. It manages concurrency with a semaphore, handles rate limits with automatic exponential backoff, and efficiently processes batches of input strings including de-duplication to avoid redundant API calls.                                                                           | Why is the dataclass frozen and how is the semaphore initialized?                        | The dataclass is frozen to make instances immutable. The semaphore is initialized in the __post_init__ method using object.__setattr__ to bypass immutability and set the _semaphore attribute.                                                                                                                    | [ 0.01004979  0.01801932  0.02163467 ...  0.011714   -0.00108944        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.02387273]                                                           |
| openaivec.aio.embeddings | AsyncBatchEmbeddings          | class               | A dataclass providing an asynchronous interface to generate embeddings using OpenAI's /embeddings endpoint. It manages concurrency with a semaphore, handles rate limits with automatic exponential backoff, and efficiently processes batches of input strings including de-duplication to avoid redundant API calls.                                                                           | What is the purpose of the observe decorator?                                            | The observe decorator (imported from openaivec.log) is used to add logging or monitoring around the decorated methods, likely to track usage, performance, or errors.                                                                                                                                              | [ 0.02522931  0.02312011 -0.05586683 ... -0.02644616 -0.02747372        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.00834217]                                                           |
| openaivec.aio.pandas_ext | use                           | function            | Registers a custom asynchronous OpenAI-compatible client to be used by the pandas extension helpers. The client must be an instance of either AsyncOpenAI or AsyncAzureOpenAI. This client is stored globally and reused for all OpenAI operations within the module.                                                                                                                            | What types of clients can be registered using the use function?                          | Only instances of openai.AsyncOpenAI or openai.AsyncAzureOpenAI can be registered. If the client is not one of these types, a TypeError is raised.                                                                                                                                                                 | [ 0.0018804   0.04456352  0.0320275  ... -0.02639303 -0.02550338        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.01136329]                                                           |
| openaivec.aio.pandas_ext | use                           | function            | Registers a custom asynchronous OpenAI-compatible client to be used by the pandas extension helpers. The client must be an instance of either AsyncOpenAI or AsyncAzureOpenAI. This client is stored globally and reused for all OpenAI operations within the module.                                                                                                                            | What happens if I call use multiple times with different clients?                        | The global client variable _CLIENT is overwritten each time use is called, so the last registered client will be used for subsequent operations.                                                                                                                                                                   | [ 0.00457617 -0.03071219  0.08553869 ... -0.04636022 -0.05345438        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.04890011]                                                           |
| openaivec.aio.pandas_ext | use_openai                    | function            | Creates and registers a default AsyncOpenAI client using the provided OpenAI API key. This function simplifies client setup by instantiating AsyncOpenAI internally and storing it globally for use by the pandas extension.                                                                                                                                                                     | What parameter does use_openai require?                                                  | It requires an api_key string which is forwarded to the AsyncOpenAI constructor.                                                                                                                                                                                                                                   | [ 0.0011076   0.02567289  0.04957882 ...  0.01267404 -0.0183582         |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.00303209]                                                           |
| openaivec.aio.pandas_ext | use_openai                    | function            | Creates and registers a default AsyncOpenAI client using the provided OpenAI API key. This function simplifies client setup by instantiating AsyncOpenAI internally and storing it globally for use by the pandas extension.                                                                                                                                                                     | Does use_openai support Azure OpenAI?                                                    | No, use_openai is specifically for the standard OpenAI API. For Azure OpenAI, use the use_azure_openai function.                                                                                                                                                                                                   | [ 0.00156495 -0.02023325  0.05007851 ... -0.01501875 -0.0027229         |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.00270488]                                                           |
| openaivec.aio.pandas_ext | use_azure_openai              | function            | Creates and registers an AsyncAzureOpenAI client using Azure-specific parameters: API key, endpoint URL, and API version. This client is stored globally for use by the pandas extension helpers.                                                                                                                                                                                                | What parameters are required to use use_azure_openai?                                    | It requires api_key (Azure subscription key), endpoint (resource endpoint URL), and api_version (REST API version string).                                                                                                                                                                                         | [-0.00958073  0.0016582   0.06137716 ... -0.02074881  0.02781625        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.0015149 ]                                                           |
| openaivec.aio.pandas_ext | use_azure_openai              | function            | Creates and registers an AsyncAzureOpenAI client using Azure-specific parameters: API key, endpoint URL, and API version. This client is stored globally for use by the pandas extension helpers.                                                                                                                                                                                                | Can I use this function with the standard OpenAI API?                                    | No, this function is specifically for Azure OpenAI deployments.                                                                                                                                                                                                                                                    | [ 0.0218661   0.00698536  0.01908477 ...  0.00088599 -0.01830293        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.00801714]                                                           |
| openaivec.aio.pandas_ext | responses_model               | function            | Overrides the model used for generating text responses from the OpenAI API. It also updates the tiktoken encoding used for token counting based on the new model name. If the model name is not recognized by tiktoken, a fallback encoding is used with a warning logged.                                                                                                                       | What is the default model name for responses?                                            | The default is 'gpt-4o-mini'.                                                                                                                                                                                                                                                                                      | [-0.02745681  0.00565146  0.04103405 ...  0.01227861  0.01833203        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.00079833]                                                           |
| openaivec.aio.pandas_ext | responses_model               | function            | Overrides the model used for generating text responses from the OpenAI API. It also updates the tiktoken encoding used for token counting based on the new model name. If the model name is not recognized by tiktoken, a fallback encoding is used with a warning logged.                                                                                                                       | What happens if the model name is not supported by tiktoken?                             | A warning is logged, and the encoding falls back to 'o200k_base'.                                                                                                                                                                                                                                                  | [-0.00128046  0.00085147  0.00921023 ...  0.02307432  0.00040928        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.03416299]                                                           |
| openaivec.aio.pandas_ext | embeddings_model              | function            | Overrides the model used for generating text embeddings from the OpenAI API. This sets the global embedding model name used by the pandas extension.                                                                                                                                                                                                                                             | What is the default embeddings model?                                                    | The default is 'text-embedding-3-small'.                                                                                                                                                                                                                                                                           | [-0.01520886 -0.01832679  0.04243724 ... -0.00143699 -0.01405451        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.00303463]                                                           |
| openaivec.aio.pandas_ext | embeddings_model              | function            | Overrides the model used for generating text embeddings from the OpenAI API. This sets the global embedding model name used by the pandas extension.                                                                                                                                                                                                                                             | Does this function affect response generation models?                                    | No, it only affects the embeddings model.                                                                                                                                                                                                                                                                          | [-0.01270008 -0.03993503  0.03164987 ... -0.00824215  0.00999092        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.03293995]                                                           |
| openaivec.aio.pandas_ext | _get_openai_client            | function            | Retrieves the globally registered OpenAI client. If no client is registered, it attempts to create one from environment variables for either OpenAI or Azure OpenAI. Raises a ValueError if no suitable credentials are found.                                                                                                                                                                   | What environment variables does _get_openai_client check for?                            | It checks for 'OPENAI_API_KEY' for standard OpenAI, and 'AZURE_OPENAI_API_KEY', 'AZURE_OPENAI_ENDPOINT', and 'AZURE_OPENAI_API_VERSION' for Azure OpenAI.                                                                                                                                                          | [-0.01753412  0.01483099  0.07935605 ... -0.01058925 -0.01251402        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.00763271]                                                           |
| openaivec.aio.pandas_ext | _get_openai_client            | function            | Retrieves the globally registered OpenAI client. If no client is registered, it attempts to create one from environment variables for either OpenAI or Azure OpenAI. Raises a ValueError if no suitable credentials are found.                                                                                                                                                                   | What happens if no client is registered and no environment variables are set?            | A ValueError is raised instructing the user to set the appropriate environment variables or register a client manually.                                                                                                                                                                                            | [-0.04086812  0.03370593  0.09328815 ... -0.04032903  0.0024628         |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.01569777]                                                           |
| openaivec.aio.pandas_ext | _extract_value                | function            | Converts a single Series element into a homogeneous dictionary representation. Supports None, Pydantic BaseModel instances (converted via model_dump), and dicts. Returns an empty dict and logs a warning if the value cannot be coerced.                                                                                                                                                       | What types of values can _extract_value handle?                                          | It can handle None, dict, and Pydantic BaseModel instances.                                                                                                                                                                                                                                                        | [0.00220707 0.04320759 0.01409346 ... 0.00263259 0.01269604 0.01965667] |
| openaivec.aio.pandas_ext | _extract_value                | function            | Converts a single Series element into a homogeneous dictionary representation. Supports None, Pydantic BaseModel instances (converted via model_dump), and dicts. Returns an empty dict and logs a warning if the value cannot be coerced.                                                                                                                                                       | What does it return if the value is not one of the supported types?                      | It returns an empty dictionary and logs a warning.                                                                                                                                                                                                                                                                 | [-0.04118536  0.03042039  0.040141   ... -0.00280338 -0.02173077        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.0616977 ]                                                           |
| openaivec.aio.pandas_ext | OpenAIVecSeriesAccessor       | class               | A pandas Series accessor registered under the name 'ai' that adds asynchronous OpenAI helper methods to pandas Series objects. It provides methods to generate text responses, compute embeddings, count tokens, and extract dictionary-like data from Series elements.                                                                                                                          | How do I use the OpenAIVecSeriesAccessor?                                                | Access it via the .ai attribute on a pandas Series, e.g., series.ai.responses(...).                                                                                                                                                                                                                                | [ 0.02481819 -0.00859039  0.03961653 ...  0.00916898 -0.00210335        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.05535424]                                                           |
| openaivec.aio.pandas_ext | OpenAIVecSeriesAccessor       | class               | A pandas Series accessor registered under the name 'ai' that adds asynchronous OpenAI helper methods to pandas Series objects. It provides methods to generate text responses, compute embeddings, count tokens, and extract dictionary-like data from Series elements.                                                                                                                          | Are the responses and embeddings methods synchronous or asynchronous?                    | They are asynchronous and must be awaited.                                                                                                                                                                                                                                                                         | [-0.03499826 -0.04006859  0.04916918 ...  0.00233203 -0.01060869        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.00756649]                                                           |
| openaivec.aio.pandas_ext | OpenAIVecSeriesAccessor       | class               | A pandas Series accessor registered under the name 'ai' that adds asynchronous OpenAI helper methods to pandas Series objects. It provides methods to generate text responses, compute embeddings, count tokens, and extract dictionary-like data from Series elements.                                                                                                                          | What does the count_tokens method do?                                                    | It counts the number of tokens in each Series element using the tiktoken encoding for the current response model.                                                                                                                                                                                                  | [ 0.00695031  0.00962914  0.01999129 ... -0.01915166 -0.01439373        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.02489582]                                                           |
| openaivec.aio.pandas_ext | OpenAIVecSeriesAccessor       | class               | A pandas Series accessor registered under the name 'ai' that adds asynchronous OpenAI helper methods to pandas Series objects. It provides methods to generate text responses, compute embeddings, count tokens, and extract dictionary-like data from Series elements.                                                                                                                          | What is the purpose of the extract method?                                               | It expands a Series of dicts or Pydantic models into a DataFrame with one column per key, optionally prefixing columns with the Series name.                                                                                                                                                                       | [-0.02733146  0.03713624 -0.03787287 ...  0.00716308 -0.0543073         |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.01781879]                                                           |
| openaivec.aio.pandas_ext | OpenAIVecDataFrameAccessor    | class               | A pandas DataFrame accessor registered under the name 'ai' that adds OpenAI helper methods to pandas DataFrames. It provides methods to extract nested dict-like columns and asynchronously generate text responses for each row serialized as JSON.                                                                                                                                             | How do I use the OpenAIVecDataFrameAccessor?                                             | Access it via the .ai attribute on a pandas DataFrame, e.g., df.ai.responses(...).                                                                                                                                                                                                                                 | [0.0430714  0.00475993 0.04101441 ... 0.02684124 0.00896798 0.04547959] |
| openaivec.aio.pandas_ext | OpenAIVecDataFrameAccessor    | class               | A pandas DataFrame accessor registered under the name 'ai' that adds OpenAI helper methods to pandas DataFrames. It provides methods to extract nested dict-like columns and asynchronously generate text responses for each row serialized as JSON.                                                                                                                                             | What does the extract method do?                                                         | It flattens a specified column containing dicts or Pydantic models into separate top-level columns, dropping the original column.                                                                                                                                                                                  | [-0.0354525   0.02179999 -0.02859872 ... -0.00533301 -0.05125199        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.0202861 ]                                                           |
| openaivec.aio.pandas_ext | OpenAIVecDataFrameAccessor    | class               | A pandas DataFrame accessor registered under the name 'ai' that adds OpenAI helper methods to pandas DataFrames. It provides methods to extract nested dict-like columns and asynchronously generate text responses for each row serialized as JSON.                                                                                                                                             | How does the responses method work?                                                      | It serializes each DataFrame row to JSON and sends it as input to the OpenAI model asynchronously, returning a Series of responses.                                                                                                                                                                                | [-0.04433542  0.02098543  0.06029617 ...  0.00080115 -0.00401741        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |  -0.01306728]                                                           |
| openaivec.aio.pandas_ext | OpenAIVecDataFrameAccessor    | class               | A pandas DataFrame accessor registered under the name 'ai' that adds OpenAI helper methods to pandas DataFrames. It provides methods to extract nested dict-like columns and asynchronously generate text responses for each row serialized as JSON.                                                                                                                                             | Are the responses method calls synchronous or asynchronous?                              | They are asynchronous and must be awaited.                                                                                                                                                                                                                                                                         | [-0.06483479 -0.02732064  0.04752654 ... -0.00221042 -0.01545841        |
|                          |                               |                     |                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                          |                                                                                                                                                                                                                                                                                                                    |   0.01356977]                                                           |